### utilities

In [1]:
import random
import math
import csv
import random
import numpy as np
import sys
import pandas as pd
import torch
from itertools import combinations

import pickle

from collections import defaultdict, Counter
from sklearn.cluster import OPTICS, cluster_optics_dbscan

import torch.nn as nn
import torch.nn.functional as F
import copy

# 20200518 
# accuracy 和 label aggregation 分开
# 计算一个 cluster 里面 worker 的平均的准确率;

def accuracy(workerlist,threshold,t2truth):
  # worker回答的 task 先找到, evaluate其他的worker和special worker的方式不一样
  # 非special worker 需要他们的准确率 只是在实验时候展示,对于special worker 选择时候是要选择5个task 进行evaluate;
  # 他不是 spammer 那么, 如果回答的task 不够覆盖全部task怎么办?  
  tasklist = []
  worker_acc_dict = {}
  for w in workerlist:
    #w2tl的结果是 tuple list
    total_tuple_list = w2tl[w]
    # 用threshold 来确定 evaluate 一个worker 用多少task

    if len(total_tuple_list) > threshold:
      short_task_list = random.sample(total_tuple_list, 5)
    # acc 先记录个数, 再计算准确率
    acc =0
    for t_id,w_answer in total_tuple_list:
      # 还需要一个truth[t_id], 可能会有不存在的情况, 如何处理
      truth = t2truth[t_id]
      if w_answer == truth:
        acc = acc+1
      
    acc = acc/len(total_tuple_list)
    worker_acc_dict[w] = acc
  # with open(path+str(threshold)+'_'+'worker_acc_dict.pickle','wb') as file:
  #   pickle.dump(worker_acc_dict,file)
  return worker_acc_dict

def accuracy_all(workerlist,t2truth):
  # worker回答的 task 先找到, evaluate其他的worker和special worker的方式不一样
  # 非special worker 需要他们的准确率 只是在实验时候展示,对于special worker 选择时候是要选择5个task 进行evaluate;
  # 他不是 spammer 那么, 如果回答的task 不够覆盖全部task怎么办?  
  # tasklist = []
  worker_acc_dict = {}
  for w in workerlist:
    #w2tl的结果是 tuple list
    total_tuple_list = w2tl[w]
    # 用threshold 来确定 evaluate 一个worker 用多少task
    #多余5个元素选择5个选择5个
    # if len(total_tuple_list) > threshold:
    #   short_task_list = random.sample(total_list, 5)
    # acc 先记录个数, 再计算准确率
    acc =0
    for t_id,w_answer in total_tuple_list:
      # 还需要一个truth[t_id], 可能会有不存在的情况, 如何处理
      truth = t2truth[t_id]
      if w_answer == truth:
        acc = acc+1
      
    acc = acc/len(total_tuple_list)
    worker_acc_dict[w] = acc
  # with open(path+'worker_acc_all_dict.pickle','wb') as file:
  #   pickle.dump(worker_acc_dict,file)
  return worker_acc_dict
def worker_cluster_acc(ep_label_woker_dict,worker_acc_dict):
  cluster_acc_dict={}
  for ep in ep_label_woker_dict:
    label_wid_dict = ep_label_woker_dict[ep]
  #special worker 的分类是-1
  for label in label_list:
    worker_list = label_wid_dict[label]
    #测试一下每一类worker的准确率 
    #acc 累加,结果是一个cluster中所有worker的 acc 平均值
    acc = 0
    for wid in worker_list:
      #worker_acc 是一个dict
      acc_w = worker_acc_dict[wid]
      acc = acc + acc_w
    acc = acc/len(worker_list)
    cluster_acc_dict[label] = acc
  
  return cluster_acc_dict

# 做一个函数 numpy，每一行的id就是worker的num_id ( 是w2input要numI_id 和str_id 对应上)
# 就是一个ppmi
def input2numpy(input_dict,wstrid2wnumid,wnumid2wstrid,feat_size,path,name):
  x_train = np.zeros((feat_size,),dtype = np.float)
  for i in range(len(input_dict)):
    #确认number 对应的string_id
    str_id  = wnumid2wstrid[i]
    #对应worker的 input array
    worker_input_array = np.asarray(input_dict[str_id])
    x_train = np.vstack((x_train,worker_input_array))

  x_train_all = np.delete(x_train,0,axis=0)
  np.save(path+name,x_train_all)
  return x_train_all

def extend_nodes(nodes_batch, training_cps, wstrid2wnumid):
  #深复制之后就是两个对象，需要import copy
  original_nodes_batch = copy.deepcopy(nodes_batch)
  node_pairs = {}
  extended_nodes_batch = set(nodes_batch)
  #wnumid2wstrid = wnumid2wstrid
  for node in nodes_batch:
      strid = wnumid2wstrid[node]
      cps = training_cps[strid]
      node_pairs[node] = cps
      for cp in cps:
        extended_nodes_batch.add(cp[1])
  extended_nodes_batch = list(extended_nodes_batch)
  return extended_nodes_batch

def w2wsim2numpy(input_dict,wstrid2wnumid,wnumid2wstrid,feat_size,path,name):
  x_train = np.zeros((feat_size,),dtype = np.float)
  for i in range(len(input_dict)):
    #确认number 对应的string_id
    str_id  = wnumid2wstrid[i]
    #对应worker的 input array
    worker_input_array = np.asarray(input_dict[str_id])

    #设定一个threshold, 小于threshold(0.3), 设置成零, 大于threshold 设置成1
    worker_input_array[worker_input_array<0.3] = 0
    worker_input_array[worker_input_array>=0.3] = 1

    x_train = np.vstack((x_train,worker_input_array))

  x_train_all = np.delete(x_train,0,axis=0)
  np.save(path+name,x_train_all)
  return x_train_all

### DeepFD

In [2]:
class DeepFD(nn.Module):
    def __init__(self, w_pb_input, feat_size, emb_size):
    #def __init__(self, features, feat_size, hidden_size, emb_size):
        super(DeepFD, self).__init__()
        self.w_pb_input = w_pb_input
        # feat_size 就是输入的size, hidden_size就是第一次变换的size,emb_size就是encoder输出的size;
        self.fc1 = nn.Linear(feat_size, 64)
        self.fc2 = nn.Linear(64, emb_size)
        self.fc3 = nn.Linear(emb_size, 64)
        self.fc4 = nn.Linear(64, feat_size)
        
    def init_params(self):
        for param in self.parameters():
            if len(param.size()) == 2:
                nn.init.xavier_uniform_(param)
            else:
                # initialize all bias as zeros
                nn.init.constant_(param, 0.0)
    # forward 的参数, nodes_batch 就是 forward 函数的 input; 
    # nodes_batch 就是一组数据,nodes_batch 是编号, features.nodes_batch 才是数据
    # def forward(self, nodes_batch):
    #     feats = self.w_pb_input[nodes_batch]
    #     x_en = F.relu_(self.fc1(feats))
    #     embs = F.relu_(self.fc2(x_en))
    #     x_de = F.relu_(self.fc3(embs))
    #     recon = F.relu_(self.fc4(x_de))
    #     #20200716 换成sigmoid 激活函数
    #     # recon = F.sigmoid(self.fc4(x_de))
    #     return embs, recon
    def forward(self, nodes_batch):
        #20201231 加上噪声
        feats = self.w_pb_input[nodes_batch]
        
        #均值是0, 方差是 1 的 高斯噪声
        #feats = feats + torch.rand(feats.shape)
        
        x_en = F.relu_(self.fc1(feats))
        embs = F.relu_(self.fc2(x_en))
        x_de = F.relu_(self.fc3(embs))
        recon = F.relu_(self.fc4(x_de))
        #20200716 换成sigmoid 激活函数
        # recon = F.sigmoid(self.fc4(x_de))
        return embs, recon

In [3]:
# 20200824 加了一个layersize
class DeepFD(nn.Module):
    def __init__(self, w_pb_input, feat_size, layersize, emb_size):
    #def __init__(self, features, feat_size, hidden_size, emb_size):
        super(DeepFD, self).__init__()
        self.w_pb_input = w_pb_input
        # feat_size 就是输入的size, hidden_size就是第一次变换的size,emb_size就是encoder输出的size;
        self.fc1 = nn.Linear(feat_size, layersize)
        self.fc2 = nn.Linear(layersize, emb_size)
        self.fc3 = nn.Linear(emb_size, layersize)
        self.fc4 = nn.Linear(layersize, feat_size)
        
    def init_params(self):
        for param in self.parameters():
            if len(param.size()) == 2:
                nn.init.xavier_uniform_(param)
            else:
                # initialize all bias as zeros
                nn.init.constant_(param, 0.0)
    # forward 的参数, nodes_batch 就是 forward 函数的 input; 
    # nodes_batch 就是一组数据,nodes_batch 是编号, features.nodes_batch 才是数据
    def forward(self, nodes_batch):
        feats = self.w_pb_input[nodes_batch]
        x_en = F.relu_(self.fc1(feats))
        embs = F.relu_(self.fc2(x_en))
        x_de = F.relu_(self.fc3(embs))
        recon = F.relu_(self.fc4(x_de))
        #20200716 换成sigmoid 激活函数
        # recon = F.sigmoid(self.fc4(x_de))
        return embs, recon

###DeepFD loss

In [4]:
# class Loss_DeepFD():
#     def __init__(self, features, worker_similarity_df, wnumid2wstrid, training_cps, device, alpha, beta, gamma):
#         self.worker_similarity_df = worker_similarity_df
#         self.wnumid2wstrid = wnumid2wstrid
        

#         self.training_cps = training_cps

#         self.features = features
        
#         self.device = device
#         #三部分的权重
#         self.alpha = alpha
#         self.beta = beta
#         self.gamma = gamma
#         #下面和simi 相关的重点关注
#         #这个graph_simi是要改一下的
#         #self.graph_simi = graph_simi
        
        
#         self.node_pairs = {}
#         self.original_nodes_batch = None
#         self.extended_nodes_batch = None
#     # def extend_nodes(self, nodes_batch, training_cps):
#     #     #深复制之后就是两个对象
#     #     self.original_nodes_batch = copy.deepcopy(nodes_batch)
#     #     self.node_pairs = {}
#     #     self.extended_nodes_batch = set(nodes_batch)

#     #     for node in nodes_batch:
#     #         cps = training_cps[node]
#     #         self.node_pairs[node] = cps
#     #         for cp in cps:
#     #             self.extended_nodes_batch.add(cp[1])
#     #     self.extended_nodes_batch = list(self.extended_nodes_batch)
    
#     # def extend_nodes(self, nodes_batch, training_cps, wnumid2wstrid):
#     #     #深复制之后就是两个对象，需要import copy
#     #     self.original_nodes_batch = copy.deepcopy(nodes_batch)
#     #     self.node_pairs = {}
#     #     self.extended_nodes_batch = set(nodes_batch)
#     #     self.wnumid2wstrid = wnumid2wstrid
#     #     for node in nodes_batch:
#     #         strid = self.wnumid2wstrid[node]
#     #         cps = training_cps[strid]
#     #         self.node_pairs[node] = cps
#     #         for cp in cps:
#     #           extended_nodes_batch.add(cp[1])
#     #     self.extended_nodes_batch = list(extended_nodes_batch)
#     #     return self.extended_nodes_batch

#     def extend_nodes(self, nodes_batch):
#         #深复制之后就是两个对象，需要import copy
#         self.original_nodes_batch = copy.deepcopy(nodes_batch)
#         self.node_pairs = {}
#         self.extended_nodes_batch = set(nodes_batch)
#         #self.wnumid2wstrid = wnumid2wstrid
#         for node in nodes_batch:
#             strid = self.wnumid2wstrid[node]
#             cps = self.training_cps[strid]
#             self.node_pairs[node] = cps
#             for cp in cps:
#               self.extended_nodes_batch.add(cp[1])
#         self.extended_nodes_batch = list(self.extended_nodes_batch)
#         return self.extended_nodes_batch
    
#     def get_loss(self, nodes_batch, embs_batch, recon_batch):
#         # calculate loss_simi and loss+recon,
#         # loss_reg is included in SGD optimizer as weight_decay
#         loss_recon = self.get_loss_recon(nodes_batch, recon_batch)
#         loss_simi = self.get_loss_simi(embs_batch)
#         loss = loss_recon + self.alpha * loss_simi
#         return loss
    
#     #def get_loss_simi(self, embs_batch, nodes_batch):
#     def get_loss_simi(self, embs_batch):
#         #node2index n 是extended_nodes 就是worker num_id; i 是 整个batch 数量的索引
#         node2index = {n:i for i,n in enumerate(self.extended_nodes_batch)}
        
#         simi_feat = []
#         simi_embs = []
#         #用nodes_batch 把nodes_pair找出来. node_pairs在extended_nodes中赋值过了
#         #self.node_pairs[node]
        
        
#         for node, cps in self.node_pairs.items():
#             for i, j in cps:
#               #graph_simi 这里需要dataframe了
#               wistrid=wnumid2wstrid[i]
#               wjstrid=wnumid2wstrid[j]
#               simi = self.worker_similarity_df.loc[wistrid,wjstrid]

#               #simi_feat.append(torch.FloatTensor([self.graph_simi[i, j]]))
#               simi_feat.append(torch.FloatTensor([simi]))
#               dis_ij = (embs_batch[node2index[i]] - embs_batch[node2index[j]]) ** 2
#               dis_ij = torch.exp(-dis_ij.sum())
#               #torch.view 好比numpy.shape/numpy.reshape
#               simi_embs.append(dis_ij.view(1))
#         simi_feat = torch.cat(simi_feat, 0).to(self.device)
#         simi_embs = torch.cat(simi_embs, 0)
#         L = simi_feat * ((simi_embs - simi_feat) ** 2)
#         return L.mean()

#     def get_loss_recon(self, nodes_batch, recon_batch):
#         feats_batch = self.features[nodes_batch]
#         # H_batch 应该就是哈德马积
#         H_batch = (feats_batch * (self.beta - 1)) + 1
#         assert feats_batch.size() == recon_batch.size() == H_batch.size()
#         #**可以用来表示幂次运算 2 ** 7  # 2 to the power of 7
#         L = ((recon_batch - feats_batch) * H_batch) ** 2
#         return L.mean()

In [5]:
#20200618
class Loss_DeepFD():
    def __init__(self, features, worker_similarity_df, wnumid2wstrid, training_cps, device, alpha, beta, gamma):
        self.worker_similarity_df = worker_similarity_df
        self.wnumid2wstrid = wnumid2wstrid
        

        self.training_cps = training_cps

        self.features = features
        
        self.device = device
        #三部分的权重
        self.alpha = alpha
        self.beta = beta
        self.gamma = gamma
        #下面和simi 相关的重点关注
        #这个graph_simi是要改一下的
        #self.graph_simi = graph_simi
        
        
        self.node_pairs = {}
        self.original_nodes_batch = None
        self.extended_nodes_batch = None
    # def extend_nodes(self, nodes_batch, training_cps):
    #     #深复制之后就是两个对象
    #     self.original_nodes_batch = copy.deepcopy(nodes_batch)
    #     self.node_pairs = {}
    #     self.extended_nodes_batch = set(nodes_batch)

    #     for node in nodes_batch:
    #         cps = training_cps[node]
    #         self.node_pairs[node] = cps
    #         for cp in cps:
    #             self.extended_nodes_batch.add(cp[1])
    #     self.extended_nodes_batch = list(self.extended_nodes_batch)
    
    # def extend_nodes(self, nodes_batch, training_cps, wnumid2wstrid):
    #     #深复制之后就是两个对象，需要import copy
    #     self.original_nodes_batch = copy.deepcopy(nodes_batch)
    #     self.node_pairs = {}
    #     self.extended_nodes_batch = set(nodes_batch)
    #     self.wnumid2wstrid = wnumid2wstrid
    #     for node in nodes_batch:
    #         strid = self.wnumid2wstrid[node]
    #         cps = training_cps[strid]
    #         self.node_pairs[node] = cps
    #         for cp in cps:
    #           extended_nodes_batch.add(cp[1])
    #     self.extended_nodes_batch = list(extended_nodes_batch)
    #     return self.extended_nodes_batch

    def extend_nodes(self, nodes_batch):
        #深复制之后就是两个对象，需要import copy
        self.original_nodes_batch = copy.deepcopy(nodes_batch)
        self.node_pairs = {}
        self.extended_nodes_batch = set(nodes_batch)
        #self.wnumid2wstrid = wnumid2wstrid
        for node in nodes_batch:
            strid = self.wnumid2wstrid[node]
            cps = self.training_cps[strid]
            self.node_pairs[node] = cps
            for cp in cps:
              self.extended_nodes_batch.add(cp[1])
        self.extended_nodes_batch = list(self.extended_nodes_batch)
        return self.extended_nodes_batch
    
    def get_loss(self, nodes_batch, embs_batch, recon_batch):
        # calculate loss_simi and loss+recon,
        # loss_reg is included in SGD optimizer as weight_decay
        loss_recon = self.get_loss_recon(nodes_batch, recon_batch)
        loss_simi = self.get_loss_simi(embs_batch)
        loss = loss_recon + self.alpha * loss_simi
        return loss
    
    #def get_loss_simi(self, embs_batch, nodes_batch):
    def get_loss_simi(self, embs_batch):
        #node2index n 是extended_nodes 就是worker num_id; i 是 整个batch 数量的索引
        node2index = {n:i for i,n in enumerate(self.extended_nodes_batch)}
        
        simi_feat = []
        simi_embs = []
        #用nodes_batch 把nodes_pair找出来. node_pairs在extended_nodes中赋值过了
        #self.node_pairs[node]
        
        
        for node, cps in self.node_pairs.items():
            for i, j in cps:
              #graph_simi 这里需要dataframe了
              wistrid=wnumid2wstrid[i]
              wjstrid=wnumid2wstrid[j]
              #20200709 针对worker id 是 int的改造
              simi = self.worker_similarity_df.loc[wistrid,wjstrid]

              #simi_feat.append(torch.FloatTensor([self.graph_simi[i, j]]))
              simi_feat.append(torch.FloatTensor([simi]))
              dis_ij = (embs_batch[node2index[i]] - embs_batch[node2index[j]]) ** 2
              dis_ij = torch.exp(-dis_ij.sum())
              #torch.view 好比numpy.shape/numpy.reshape
              simi_embs.append(dis_ij.view(1))
        simi_feat = torch.cat(simi_feat, 0).to(self.device)
        simi_embs = torch.cat(simi_embs, 0)
        L = simi_feat * ((simi_embs - simi_feat) ** 2)
        return L.mean()

    def get_loss_recon(self, nodes_batch, recon_batch):
        feats_batch = self.features[nodes_batch]
        # H_batch 应该就是哈德马积
        H_batch = (feats_batch * (self.beta - 1)) + 1
        assert feats_batch.size() == recon_batch.size() == H_batch.size()
        #**可以用来表示幂次运算 2 ** 7  # 2 to the power of 7
        L = ((recon_batch - feats_batch) * H_batch) ** 2
        return L.mean()

In [6]:
#20200709 
# 针对wid是整数的操作;
class Loss_DeepFD():
    def __init__(self, features, worker_similarity_df, wnumid2wstrid, training_cps, device, alpha, beta, gamma):
        self.worker_similarity_df = worker_similarity_df
        self.wnumid2wstrid = wnumid2wstrid
        

        self.training_cps = training_cps

        self.features = features
        
        self.device = device
        #三部分的权重
        self.alpha = alpha
        self.beta = beta
        self.gamma = gamma
        #下面和simi 相关的重点关注
        #这个graph_simi是要改一下的
        #self.graph_simi = graph_simi
        
        
        self.node_pairs = {}
        self.original_nodes_batch = None
        self.extended_nodes_batch = None
    # def extend_nodes(self, nodes_batch, training_cps):
    #     #深复制之后就是两个对象
    #     self.original_nodes_batch = copy.deepcopy(nodes_batch)
    #     self.node_pairs = {}
    #     self.extended_nodes_batch = set(nodes_batch)

    #     for node in nodes_batch:
    #         cps = training_cps[node]
    #         self.node_pairs[node] = cps
    #         for cp in cps:
    #             self.extended_nodes_batch.add(cp[1])
    #     self.extended_nodes_batch = list(self.extended_nodes_batch)
    
    # def extend_nodes(self, nodes_batch, training_cps, wnumid2wstrid):
    #     #深复制之后就是两个对象，需要import copy
    #     self.original_nodes_batch = copy.deepcopy(nodes_batch)
    #     self.node_pairs = {}
    #     self.extended_nodes_batch = set(nodes_batch)
    #     self.wnumid2wstrid = wnumid2wstrid
    #     for node in nodes_batch:
    #         strid = self.wnumid2wstrid[node]
    #         cps = training_cps[strid]
    #         self.node_pairs[node] = cps
    #         for cp in cps:
    #           extended_nodes_batch.add(cp[1])
    #     self.extended_nodes_batch = list(extended_nodes_batch)
    #     return self.extended_nodes_batch

    def extend_nodes(self, nodes_batch):
        #深复制之后就是两个对象，需要import copy
        self.original_nodes_batch = copy.deepcopy(nodes_batch)
        self.node_pairs = {}
        self.extended_nodes_batch = set(nodes_batch)
        #self.wnumid2wstrid = wnumid2wstrid
        for node in nodes_batch:
            strid = self.wnumid2wstrid[node]
            cps = self.training_cps[strid]
            self.node_pairs[node] = cps
            for cp in cps:
              self.extended_nodes_batch.add(cp[1])
        self.extended_nodes_batch = list(self.extended_nodes_batch)
        return self.extended_nodes_batch
    
    def get_loss(self, nodes_batch, embs_batch, recon_batch):
        # calculate loss_simi and loss+recon,
        # loss_reg is included in SGD optimizer as weight_decay
        loss_recon = self.get_loss_recon(nodes_batch, recon_batch)
        loss_simi = self.get_loss_simi(embs_batch)
        loss = loss_recon + self.alpha * loss_simi
        return loss
    
    #def get_loss_simi(self, embs_batch, nodes_batch):
    def get_loss_simi(self, embs_batch):
        #node2index n 是extended_nodes 就是worker num_id; i 是 整个batch 数量的索引
        node2index = {n:i for i,n in enumerate(self.extended_nodes_batch)}
        
        simi_feat = []
        simi_embs = []
        #用nodes_batch 把nodes_pair找出来. node_pairs在extended_nodes中赋值过了
        #self.node_pairs[node]
        
        
        for node, cps in self.node_pairs.items():
            for i, j in cps:
              #graph_simi 这里需要dataframe了
              wi=wnumid2wstrid[i]
              wj=wnumid2wstrid[j]
              #20200709 针对worker id 是 int的改造
              wistrid = 'w' + str(wi)
              wjstrid = 'w' + str(wj)
              simi = self.worker_similarity_df.loc[wistrid,wjstrid]

              #simi_feat.append(torch.FloatTensor([self.graph_simi[i, j]]))
              simi_feat.append(torch.FloatTensor([simi]))
              dis_ij = (embs_batch[node2index[i]] - embs_batch[node2index[j]]) ** 2
              dis_ij = torch.exp(-dis_ij.sum())
              #torch.view 好比numpy.shape/numpy.reshape
              simi_embs.append(dis_ij.view(1))
        simi_feat = torch.cat(simi_feat, 0).to(self.device)
        simi_embs = torch.cat(simi_embs, 0)
        L = simi_feat * ((simi_embs - simi_feat) ** 2)
        return L.mean()

    def get_loss_recon(self, nodes_batch, recon_batch):
        feats_batch = self.features[nodes_batch]
        # H_batch 应该就是哈德马积
        H_batch = (feats_batch * (self.beta - 1)) + 1
        assert feats_batch.size() == recon_batch.size() == H_batch.size()
        #**可以用来表示幂次运算 2 ** 7  # 2 to the power of 7
        L = ((recon_batch - feats_batch) * H_batch) ** 2
        return L.mean()

### train 函数



In [7]:
# 返回 deepFD

def train_model(wnumid2wstrid,training_cps, deepFD, model_loss, device, epoch):
  #train_model(Dl, args, logger, deepFD, model_loss, device, epoch):
    #读一下 wnumid2wstrid 字典
    #读一下 training_cps

    # D1.ds+'_train' =  np.arange(np.shape(graph_u2p)[0])
    #train_nodes = getattr(Dl, Dl.ds+'_train')
    train_nodes = np.arange(len(wnumid2wstrid))

    # 这一步把 user id 打乱
    np.random.shuffle(train_nodes)

    #存的是所有参数, 干啥呢
    params = []
    #learning rate, gama 是一个权重吗？按照原来的代码设定一下
    learning_rate = 0.025
    
    gama = 0.001
    # 参数为什么要手动更新呢?
    for param in deepFD.parameters():
        if param.requires_grad:
            params.append(param)
    
    #对参数的更新是自己写的,具体的还得再研究,其他方法的参数是怎么优化的,为什么要把参数一些参数拿出来?
    # gama 不知道写的是啥,
    optimizer = torch.optim.SGD(params, lr=lr, weight_decay=gamma)
    #下面两个都有什么区别optimizer 和deepFD 有什么区别呢?
    optimizer.zero_grad()
    deepFD.zero_grad()
    
    # b_sz 默认是 100, batch size的缩写 batches 就是一个整数, 176 = 2*2*2*11
    b_sz = 22
    batches = math.ceil(len(train_nodes) / b_sz)

    #不知道是干啥,谁访问的点? 这里没用上
    visited_nodes = set()
    
    ### 弄这个地方
    #training_cps是一个字典,存的是一个点和他的邻居, 用的是defaultdict ,key:node_id ; value : [(id,pos_id/neg_id),]
    # training_cps = Dl.get_train()
    training_cps = training_cps
    
    #logger.info('sampled pos and neg nodes for each node in this epoch.')
    for index in range(batches):
        #弄出来一个batch,还是id,不是表示
        # 这两个nodes batch 有什么区别
        # 根据数量得到的一组,比如(0,100)
        nodes_batch = train_nodes[index*b_sz:(index+1)*b_sz]
        
        
        # nodes_batch = np.asarray(model_loss.extend_nodes(nodes_batch, training_cps))
        nodes_batch = np.asarray(model_loss.extend_nodes(nodes_batch))
        
        
        # 用asarray把list 转换成array了，nodes_batch 就是entended nodes batch；
        #nodes_batch = np.asarray(Loss_DeepFD.extend_nodes(nodes_batch,training_cps))
        #nodes_batch = np.asarray(model_loss.extend_nodes(nodes_batch))
        
        # |= 取了一个并集
        visited_nodes |= set(nodes_batch)

        #deepFD(nodes_batch) 会把参数读到 forward 里面去执行,对feats 进行赋值,才是真正的 representation
        # 需要两个 model 实例,只需要一个
        # 
        # PB_embs_batch, PB_recon_batch = PB_model(nodes_batch)
        # AB_embs_batch, AB_recon_batch = AB_model(nodes_batch)

        # pb_embedding, ab_embedding, embedding, pb_recons, ab_recons = deepSW(nodes_batch)
        
        embs_batch, recon_batch = deepFD(nodes_batch)
        loss = model_loss.get_loss(nodes_batch, embs_batch, recon_batch)

        # embeddings, pb_embedding, ab_embedding, pb_recons, ab_recons = deepSW(nodes_batch)
        
        # loss = model_loss.get_loss(nodes_batch, pb_recons, ab_recons)

        # logger.info(f'EP[{epoch}], Batch [{index+1}/{batches}], Loss: {loss.item():.4f}, Dealed Nodes [{len(visited_nodes)}/{len(train_nodes)}]')
        loss.backward()

        nn.utils.clip_grad_norm_(deepFD.parameters(), 5)
        optimizer.step()

        optimizer.zero_grad()
        deepFD.zero_grad()

        #输入一下loss
        print('epoch', epoch,'batch',index, 'loss =',loss.data.numpy())
        # stop when all nodes are trained
        if len(visited_nodes) == len(train_nodes):
            break

    return deepFD 

#### get_embeddings

In [8]:
#返回一个worker_emb_numpy
def get_embeddings(deepFD, wnumid2wstrid, path):
  train_nodes = np.arange(len(wnumid2wstrid))
  with torch.no_grad():
    embs_batch, _ = deepFD(train_nodes)
  assert len(embs_batch) == len(train_nodes)
  embs_temp = embs_batch.detach()
  worker_emb_numpy = embs_temp.numpy()
  np.save(path+'worker_emb_numpy.npy',worker_emb_numpy)
  
  return worker_emb_numpy
    
    



### 选expert的方法


#### distance_experts
20200701 通过算距离的方式, 选出experts
距离越大, ability 越低;

In [9]:
# from sklearn.metrics import pairwise_distances
# pairwise_distances(a,metric="cosine")
# dists = pairwise_distances(worker_emb_numpy[i].reshape(1,-1), worker_emb_numpy[j].reshape(1,-1))

In [10]:
from sklearn.metrics import pairwise_distances
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

## scipy 也有一些算距离的公式
# from scipy.spatial import distance_matrix

from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from itertools import cycle
from sklearn.cluster import OPTICS, cluster_optics_dbscan
def distance_experts(wstrid2wnumid, wnumid2wstrid, worker_acc_dict, t2w, t2wl, w2t, path, deepSW):
  
  #读入需要分类的数据
  worker_emb_numpy = get_embeddings(deepSW, wnumid2wstrid, path)
  w2sumdistance_dict = {}
  # worker_emb_numpy = worker_emb_numpy[worker_numid_list]

  ### 20200701 计算每一个worker的degree
  # 最后用一个字典把结果存上, key 用str_id 还是 num id ? 先用num_id
  # 可以不用pandas
  # 20200701 可以和optics 最远的距离, 和我们求和的方式比较一下 experts
  for i in range(worker_emb_numpy.shape[0]):
    temp = 0
    for j in range(worker_emb_numpy.shape[0]):
      # 20200702 加上reshape 操作拉成1维
      # dists = pairwise_distances(worker_emb_numpy[i].reshape(1,-1), worker_emb_numpy[j].reshape(1,-1))
      # 20200716 cosinsimilarity 试一次
      dists = 1 - cosine_similarity(worker_emb_numpy[i].reshape(1,-1), worker_emb_numpy[j].reshape(1,-1))
      temp = temp + dists
    # 20200702 temp[0][0] 是数值 temp[0] 是 shape (1,) 的数组, temp 是shape[1,1] 的数组
    w2sumdistance_dict[i] = temp[0][0]
  
  # text = path + 'LLA-Input_worker_distance_acc_num.txt'
  # with open(text, 'wt') as f:
  #   print('-----------------------','worker_dbscan_cluster_acc', file=f)

  
  return w2sumdistance_dict

  
  

### Top1_experts
aggregation 方法

In [11]:
# 20200702 选出top1的worker 作为结果;
def take3rd(elem):
    return elem[2]
def Top1_experts(w2sumdistance_dict,wstrid2wnumid, wnumid2wstrid, t2wl):

  #t2wl = self.t2wl
  # worker_list = []
  # for i in idnumpy:
  #   strid = wnumid2wstrid[i]
  #   worker_list.append(strid)
  
  # key : task_id, value: wnumid,label,distance
  t2pwld = {}
  for t in t2wl:
    wl_list = t2wl[t]
    w1d_list = []
    predict = 0
    
    for wl_tuple in wl_list:
      worker = wl_tuple[0]
      label = wl_tuple[1]
      numid = wstrid2wnumid[worker]
      w_dis = w2sumdistance_dict[numid]
      w1d_list.append([worker,label,w_dis])
    # 根据 sum_distance  升序排列;
    w1d_list.sort(key=take3rd,reverse=False)
    # 取排在第一位的 wld 信息
    # 20200716 取几个直接进行切片就行了
    t2pwld[t] = w1d_list[0:1]

  return t2pwld
  

In [12]:
# 2020随机一个结果看看;

In [13]:
aaa = [5,2,3,1,4]
aaa.sort()
print(aaa)
bbb = aaa[:3]
print(bbb)

[1, 2, 3, 4, 5]
[1, 2, 3]


In [14]:
# 把wnum_id 转成str_id
def wstr_2_distance(w2sumdistance_dict,wstrid2wnumid):

  # key : wstr_id, value: distance
  wstr2distance = {}
  for wnumid in w2sumdistance_dict:
    wstrid = wnumid2wstrid[wnumid]
    wstr2distance[wstrid] = w2sumdistance_dict[wnumid]

  return wstr2distance

In [15]:
##20200702
# 为 top_1 设计的方法;
def getaccuracy_Top1(e2truth, t2pwld, t2w):
  #label 应该是字符串
  count = 0
  right_taskid_list = []
  wrong_taskid_list = []
  # truth = ''
  
  for e in t2w:

    ## 20200627 加上这一句就可以用全部数据集了;
    if e not in e2truth:
      continue
    # print(e)
    temp = t2pwld[e]
    #temp 的第二位表示 label
    predict_answer = temp[0][1]
  
    count += 1 
    
    if predict_answer == e2truth[e]:
      
      right_taskid_list.append(e)
    else:
      wrong_taskid_list.append(e)  

    # 准确率, 回答正确的list, 回答错误的list
  return len(right_taskid_list)/count, right_taskid_list, wrong_taskid_list

### Similar3 方法
20200708 在回答 task i 的 worker 中找到 3 个 最相似的worker 做MV

In [16]:
from sklearn.metrics import pairwise_distances
import numpy as np
from itertools import combinations

## scipy 也有一些算距离的公式
# from scipy.spatial import distance_matrix

from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from itertools import cycle
from sklearn.cluster import OPTICS, cluster_optics_dbscan


#20200709 
def S3_experts(wstrid2wnumid, wnumid2wstrid,  t2w, t2wl, w2t, path, deepSW):
  
  #读入需要分类的数据
  worker_emb_numpy = get_embeddings(deepSW, wnumid2wstrid, path)
  # 类似t2wl的, 直接用mv 就行了;
  t2s3w = {}
  t2s3wl = {}
  
  for t in t2w:
    w_list = t2w[t]
    # 把距离都算一遍
    # temp = 0
    temp = 10000
    for w_set in combinations(w_list,3):
      dists = 0
      #计算一组3worker 之间的距离
      for cal_distance in combinations(w_set,2):
        wi = cal_distance[0]
        wj = cal_distance[1]      
        numid_i = wstrid2wnumid[wi]
        numid_j = wstrid2wnumid[wj]

        dist = pairwise_distances(worker_emb_numpy[numid_i].reshape(1,-1), worker_emb_numpy[numid_j].reshape(1,-1))
        dists = dists + dist
      if dists < temp:
        temp = dist
        s3workers = w_set
    t2s3w[t] = s3workers
  # 弄一个t2wl 类似的东西;
  for t in t2wl:
    wl_list = t2wl[t]
    s3wl_list = []
    for w,l in wl_list:
      if w in t2s3w[t]:
        s3wl_list.append([w,l])
    t2s3wl[t] = s3wl_list

      


  
  return t2s3wl

In [17]:
# 20200720 有一些问题 
#from sklearn.metrics import pairwise_distances
# import numpy as np
# from itertools import combinations

# ## scipy 也有一些算距离的公式
# # from scipy.spatial import distance_matrix

# from sklearn.manifold import TSNE
# import matplotlib.pyplot as plt
# import matplotlib.colors as colors
# from itertools import cycle
# from sklearn.cluster import OPTICS, cluster_optics_dbscan


# # 20200702 选出top1的worker 作为结果;
# def take3rd(elem):
#     return elem[2]
# def Top1_experts(w2sumdistance_dict,wstrid2wnumid, wnumid2wstrid, t2wl):

#   #t2wl = self.t2wl
#   # worker_list = []
#   # for i in idnumpy:
#   #   strid = wnumid2wstrid[i]
#   #   worker_list.append(strid)
  
#   # key : task_id, value: wnumid,label,distance
#   t2pwld = {}
#   for t in t2wl:
#     wl_list = t2wl[t]
#     w1d_list = []
#     predict = 0
    
#     for wl_tuple in wl_list:
#       worker = wl_tuple[0]
#       label = wl_tuple[1]
#       numid = wstrid2wnumid[worker]
#       w_dis = w2sumdistance_dict[numid]
#       w1d_list.append([worker,label,w_dis])
#     # 根据 sum_distance  升序排列;
#     w1d_list.sort(key=take3rd,reverse=False)
#     # 取排在第一位的 wld 信息
#     # 20200716 取几个直接进行切片就行了
#     t2pwld[t] = w1d_list[0:3]

#   return t2pwld

# # 20200612 为甚要用类写, 不这么写了;

# def W_AggMethod(t2wl,label_set):

#   #t2wl = self.t2wl
#   t2lpd={}
#   for t in t2wl:
#     t2lpd[t]={}

#     # multi label 初始化;
#     for label in label_set:
#       t2lpd[t][label] = 0
#     # e2lpd[e]['0']=0
#     # e2lpd[e]['1']=0
    
#     alls = len(t2wl[t])
#     for i in range(alls):
#       # 按顺序*乘一个权重;  第一个的一票顶2票, 或者一票顶3票?


#     for item in t2wl[t]:
#       label=item[1]
#       t2lpd[t][label]+= 1

    
#     if alls!=0:
#         # e2lpd[e]['0']=1.0*e2lpd[e]['0']/alls
#         # e2lpd[e]['1']=1.0*e2lpd[e]['1']/alls
#       for label in label_set:
#           t2lpd[t][label] = 1.0 * t2lpd[t][label] / alls
#     else:
#         # e2lpd[e]['0']=0.5
#         # e2lpd[e]['1']=0.5
#       for label in label_set:
#           t2lpd[t][label] = 1.0 / len(label_set)

#   # return self.expand(e2lpd)
#   return t2lpd



#### S3 MV 方法

In [18]:
# 20200612 为甚要用类写, 不这么写了;

def AggMethod(t2wl,label_set):

  #t2wl = self.t2wl
  t2lpd={}
  for t in t2wl:
    t2lpd[t]={}

    # multi label 初始化;
    for label in label_set:
      t2lpd[t][label] = 0
    # e2lpd[e]['0']=0
    # e2lpd[e]['1']=0
    
    alls = len(t2wl[t])
    for item in t2wl[t]:
      label=item[1]
      t2lpd[t][label]+= 1

    
    if alls!=0:
        # e2lpd[e]['0']=1.0*e2lpd[e]['0']/alls
        # e2lpd[e]['1']=1.0*e2lpd[e]['1']/alls
      for label in label_set:
          t2lpd[t][label] = 1.0 * t2lpd[t][label] / alls
    else:
        # e2lpd[e]['0']=0.5
        # e2lpd[e]['1']=0.5
      for label in label_set:
          t2lpd[t][label] = 1.0 / len(label_set)

  # return self.expand(e2lpd)
  return t2lpd

In [19]:
##20200702

def getaccuracy(e2truth, e2lpd, t2w):
  # 需要确保e2lpd 已经是一个完整结果了; 
  #label 应该是字符串
  count = 0
  right_taskid_list = []
  wrong_taskid_list = []
  # truth = ''
  
  for e in t2w:

    ## 20200627 加上这一句就可以用全部数据集了;
    if e not in e2truth:
      continue
    # print(e)
    temp = 0
    predict_answer = 0
    #找到worker的mv-answer e2lpd 就是各种方法的结果; temp 存了一个值
    for label in e2lpd[e]:
      if temp < e2lpd[e][label]:
        temp = e2lpd[e][label]
        #print(temp)
      
    count += 1 
    for label in e2lpd[e]:
      if temp == e2lpd[e][label]:
        predict_answer = label
      # print('temp',temp)
      # print('predict',predict_answer)
    
    if predict_answer == e2truth[e]:
      
      right_taskid_list.append(e)
    else:
      wrong_taskid_list.append(e)  

    # 准确率, 回答正确的list, 回答错误的list
  return len(right_taskid_list)/count, right_taskid_list, wrong_taskid_list

## weighted_mv
20200720 1_weight = sum(w_ti);

In [20]:
# 20201026 返回根据距离排序的list；
# 20200702 选出top1的worker 作为结果;

# 取出第三个元素, 在下面事一个排序的key
def take3rd(elem):
    return elem[2]
# 20200720 对worker 按照distance 排序了 worker 作为结果;
def Sort_experts(w2sumdistance_dict,wstrid2wnumid, wnumid2wstrid, t2wl):

  #t2wl = self.t2wl
  # worker_list = []
  # for i in idnumpy:
  #   strid = wnumid2wstrid[i]
  #   worker_list.append(strid)
  
  # key : task_id, value: wnumid,label,distance
  t2pwld = {}
  for t in t2wl:
    wl_list = t2wl[t]
    w1d_list = []
    predict = 0
    
    for wl_tuple in wl_list:
      worker = wl_tuple[0]
      label = wl_tuple[1]
      numid = wstrid2wnumid[worker]
      w_dis = w2sumdistance_dict[numid]
      w1d_list.append([worker,label,w_dis])
    # 根据 sum_distance  升序排列;
    w1d_list.sort(key=take3rd,reverse=False)
    # 取排在第一位的 wld 信息
    # 20200716 取几个直接进行切片就行了, 不需要进行切片了;
    t2pwld[t] = w1d_list

  return t2pwld

In [21]:
def my_sum(i):
  if i < 0:
      raise ValueError
  elif i <= 1:
      return i
  else:
      return i + my_sum(i-1)

# 20200612 为甚要用类写, 不这么写了;

def W_MV_AggMethod(t2wl,label_set):

  #t2wl = self.t2wl
  t2lpd={}
  for t in t2wl:
    t2lpd[t]={}

    # multi label 初始化;
    for label in label_set:
      t2lpd[t][label] = 0
    # e2lpd[e]['0']=0
    # e2lpd[e]['1']=0
    
    # alls = len(t2wl[t])
    # for item in t2wl[t]:
    #   label=item[1]
    #   t2lpd[t][label]+= 1

    #20200720 加上一个weight;
    wld_list = t2wl[t]
    alls = len(wld_list)
    weight_sum = my_sum(alls)
    
    for i in range(alls):
      label = wld_list[i][1]
      weight = 1.0*(alls-i)
      t2lpd[t][label] += weight


    # for item in t2wl[t]:
    #   label=item[1]
    #   t2lpd[t][label]+= 1
    
    if alls!=0:
        # e2lpd[e]['0']=1.0*e2lpd[e]['0']/alls
        # e2lpd[e]['1']=1.0*e2lpd[e]['1']/alls
      for label in label_set:
          #20200720 weight_sum
          t2lpd[t][label] = 1.0 * t2lpd[t][label] / weight_sum
    else:
        # e2lpd[e]['0']=0.5
        # e2lpd[e]['1']=0.5
      for label in label_set:
          t2lpd[t][label] = 1.0 / len(label_set)

  # return self.expand(e2lpd)
  return t2lpd

In [22]:

##20200702

def W_MV_getaccuracy(e2truth, e2lpd, t2w):
  # 需要确保e2lpd 已经是一个完整结果了; 
  #label 应该是字符串
  count = 0
  right_taskid_list = []
  wrong_taskid_list = []
  # truth = ''
  
  for e in t2w:

    ## 20200627 加上这一句就可以用全部数据集了;
    if e not in e2truth:
      continue
    # print(e)
    temp = 0
    predict_answer = 0
    #找到worker的mv-answer e2lpd 就是各种方法的结果; temp 存了一个值
    for label in e2lpd[e]:
      if temp < e2lpd[e][label]:
        temp = e2lpd[e][label]
        #print(temp)
      
    count += 1 
    for label in e2lpd[e]:
      if temp == e2lpd[e][label]:
        predict_answer = label
      # print('temp',temp)
      # print('predict',predict_answer)
    
    if predict_answer == e2truth[e]:
      
      right_taskid_list.append(e)
    else:
      wrong_taskid_list.append(e)  

    # 准确率, 回答正确的list, 回答错误的list
  return len(right_taskid_list)/count, right_taskid_list, wrong_taskid_list

In [ ]:
## 20201026 运行之前所有代码;

In [ ]:
import random
import math
import csv
import random
import numpy as np
import sys
import pandas as pd
import torch
from itertools import combinations

import pickle

from collections import defaultdict, Counter
from sklearn.cluster import OPTICS, cluster_optics_dbscan

import torch.nn as nn
import torch.nn.functional as F
import copy


## main 函数
Top1 和 WMV

In [26]:
#20201231 输出文件
save_path = '/content/drive/MyDrive/Colab Notebooks/Deep worker cluster/files_for_IJCAI2021/data/'
pathlist = [#save_path + 'active-crowd-toolkit/CF/',
        # save_path + 'active-crowd-toolkit/CF_amt/',
        save_path +'active-crowd-toolkit/MS/',
        #  save_path +'active-crowd-toolkit/SP/',
        #  save_path +'active-crowd-toolkit/SP_amt/',
        #  save_path +'active-crowd-toolkit/ZenCrowd_all/',
        #  save_path +'active-crowd-toolkit/ZenCrowd_in/',
         save_path +'active-crowd-toolkit/ZenCrowd_us/',
         save_path +'SpectralMethodsMeetEM/bluebird/',
        #  save_path +'SpectralMethodsMeetEM/dog/',
        #  save_path +'SpectralMethodsMeetEM/rte/',
         save_path +'SpectralMethodsMeetEM/trec/',
        #  save_path +'SpectralMethodsMeetEM/web/',
         save_path +'crowdscale2013/fact_eval/',
        #  save_path +'crowdscale2013/sentiment/',
         save_path +'crowd_truth_inference/d_Duck Identification/',
        #  save_path +'crowd_truth_inference/d_jn-product/',
        #  save_path +'crowd_truth_inference/d_sentiment/',
        #  save_path +'crowd_truth_inference/s4_Dog data/',
         save_path +'crowd_truth_inference/s4_Face Sentiment Identification/',
        #  save_path +'crowd_truth_inference/s4_Relevance/',
        #  save_path +'crowd_truth_inference/s5_AdultContent/'
          ]

In [27]:
#20201026 输出文件
# pathlist = ['/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/data_at_scale/active-crowd-toolkit/CF/',
#         '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/data_at_scale/active-crowd-toolkit/CF_amt/',
#         '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/data_at_scale/active-crowd-toolkit/MS/',
#          '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/data_at_scale/active-crowd-toolkit/SP/',
#          '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/data_at_scale/active-crowd-toolkit/SP_amt/',
#          '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/data_at_scale/active-crowd-toolkit/ZenCrowd_all/',
#          '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/data_at_scale/active-crowd-toolkit/ZenCrowd_in/',
#          '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/data_at_scale/active-crowd-toolkit/ZenCrowd_us/',
#          '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/data_at_scale/SpectralMethodsMeetEM/bluebird/',
#          '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/data_at_scale/SpectralMethodsMeetEM/dog/',
#          '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/data_at_scale/SpectralMethodsMeetEM/rte/',
#          '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/data_at_scale/SpectralMethodsMeetEM/trec/',
#          '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/data_at_scale/SpectralMethodsMeetEM/web/',
#          '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/data_at_scale/crowdscale2013/fact_eval/',
#          '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/data_at_scale/crowdscale2013/sentiment/',
#          '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/data_at_scale/crowd_truth_inference/d_Duck Identification/',
#          '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/data_at_scale/crowd_truth_inference/d_jn-product/',
#          '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/data_at_scale/crowd_truth_inference/d_sentiment/',
#          '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/data_at_scale/crowd_truth_inference/s4_Dog data/',
#          '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/data_at_scale/crowd_truth_inference/s4_Face Sentiment Identification/',
#          '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/data_at_scale/crowd_truth_inference/s4_Relevance/',
#          '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/data_at_scale/crowd_truth_inference/s5_AdultContent/'
#           ]
# pathlist = ['Notebooks/Deep worker cluster/datasets/data_at_scale/SpectralMethodsMeetEM/trec/'


headers = ['alpha','layer','emb',
           'epoch_10_Top1_ACC','epoch_10_Top1_ACC_Right','epoch_10_Top1_ACC_Wrong','epoch_10_MV_ACC','epoch_10_MV_ACC_Right','epoch_10_MV_ACC_Wrong',
           'epoch_20_Top1_ACC','epoch_20_Top1_ACC_Right','epoch_20_Top1_ACC_Wrong','epoch_20_MV_ACC','epoch_20_MV_ACC_Right','epoch_20_MV_ACC_Wrong',
           'epoch_30_Top1_ACC','epoch_30_Top1_ACC_Right','epoch_30_Top1_ACC_Wrong','epoch_30_MV_ACC','epoch_30_MV_ACC_Right','epoch_30_MV_ACC_Wrong',
           'epoch_40_Top1_ACC','epoch_40_Top1_ACC_Right','epoch_40_Top1_ACC_Wrong','epoch_40_MV_ACC','epoch_40_MV_ACC_Right','epoch_40_MV_ACC_Wrong',
           'epoch_50_Top1_ACC','epoch_50_Top1_ACC_Right','epoch_50_Top1_ACC_Wrong','epoch_50_MV_ACC','epoch_50_MV_ACC_Right','epoch_50_MV_ACC_Wrong',
           'Top1_ACC_AV','Top1_ACC_MAX','MV_ACC_AV','MV_ACC_MAX']

output_path = '/content/drive/MyDrive/Colab Notebooks/Deep worker cluster/files_for_IJCAI2021/'
for path in pathlist:

  datasetname = path.split('/')[-2]
  # result_file = output_path + datasetname+ '_noise_WORT_TOP1_MV_answer_3.csv'
  result_file = output_path + datasetname+ '_noise_a_0_WORT_TOP1_MV_answer_3.csv'
  with open(result_file,'w')as f:
    f_csv = csv.writer(f)
    f_csv.writerow(headers)

In [ ]:
# device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [28]:
#20200824 对alpha 值的改变的测试
# 下面的值已经不全了, 因为有的数据集出了一些问题
# alpha_list = [0]
# emb_size_list= [10]
# layer_size_list = [64]
# alpha_list = [10,20,30,40,50]
alpha_list = [0]
emb_size_list= [2,3,4,5,6,7,8,9,10]
layer_size_list = [16,32,64]
output_path = '/content/drive/MyDrive/Colab Notebooks/Deep worker cluster/files_for_IJCAI2021/'


for path in pathlist:
  seed = 1234
  device = torch.device('cpu')
  # 这几个要调整一下的;
  b_sz = 10
  hidden_size = 128
  emb_size = 4
  epochs = 50
  # Hyper parameters
  # 20200824 改一下alpha 参数试一试;
  # alpha = apa
  beta = 20
  gamma = 0.01
  lr = 0.025

  # datafile = path+'answer_3.csv'
  # datafile = path + 'answer_mv_failed.csv'
  # datafile =  path + 'select_for_new_2.csv'
  ########## pb + ab 模式的input
  with open(path+'w2task_input.pickle','rb') as file:
    w2task_input = pickle.load(file)

  # with open(path+'w2w_input.pickle','rb') as file:
  #   w2w_input = pickle.load(file) 
  #######################################################################
  with open(path+'wnumid2wstrid.pickle','rb') as file:
    wnumid2wstrid = pickle.load(file)

  with open(path+'wstrid2wnumid.pickle','rb') as file:
    wstrid2wnumid = pickle.load(file)

  with open(path+'t2w.pickle','rb') as file:
    t2w = pickle.load(file)
  with open(path+'t2wl.pickle','rb') as file:
    t2wl = pickle.load(file)


  with open(path+'t2truth.pickle','rb') as file:
    t2truth = pickle.load(file)


  with open(path+'w2t.pickle','rb') as file:
    w2t = pickle.load(file)

  with open(path+'w2tl.pickle','rb') as file:
    w2tl = pickle.load(file)

  with open(path+'worker_acc_dict.pickle','rb') as file:
    worker_acc_dict = pickle.load(file)



  with open(path+'w2w.pickle','rb') as file:
    w2w = pickle.load(file) 

  with open(path+'training_cps.pickle','rb') as file:
    training_cps = pickle.load(file) 


  with open(path+'label_set.pickle','rb') as file:
    label_set = pickle.load(file)

  # dataset = path[78:]
  # rows = []
  csv_data = pd.read_csv(path+'worker_simlarity.csv',low_memory = False,index_col=0)
  worker_similarity_df = pd.DataFrame(csv_data)

  datasetname = path.split('/')[-2]
  output_file = output_path + datasetname+ '_noise_WORT_TOP1_MV_answer_3.csv'
  

  for apa in alpha_list:
    
    for layer in layer_size_list:
      for emb_size in emb_size_list:
    
        #20201026
        ##
        
        output_list =[]
        output_list.append(apa)
        output_list.append(layer)
        output_list.append(emb_size)

          
        # 20200824 改一下alpha 参数试一试;从最外层循环读进来的
        alpha = apa  

        # model 构建和训练
        pb_feat_size = len(t2w)
        w_pb_input = input2numpy(w2task_input,wstrid2wnumid,wnumid2wstrid,pb_feat_size,path,'w_pb_input.npy')
        w_pb_input_tensor = torch.FloatTensor(w_pb_input).to(device)
        # 20200824 DeepFD 函数也改了;
        deepFD = DeepFD(w_pb_input_tensor, w_pb_input_tensor.size(1),layer, emb_size)
        deepFD.to(device)
        #应该是在这才循环apha
        model_loss = Loss_DeepFD(w_pb_input_tensor, worker_similarity_df, wnumid2wstrid, training_cps, device, alpha, beta, gamma)

        # 需要写下来的东西, 保存起来的;
        
        ####20200811多一个max指标
        max_temp_Top1 = 0
        max_temp_MV = 0

        acc_Top1_all = 0
        acc_WM_all = 0

        for epoch in range(epochs):
          #logger.info(f'----------------------EPOCH {epoch}-----------------------')
          
          deepFD = train_model(wnumid2wstrid, training_cps, deepFD, model_loss, device, epoch)
          #deepSW = train_model_DeepSW(wnumid2wstrid, deepSW, model_loss, device, epoch)
          
          
          #每训练10次出一个结果;
          if (epoch+1) % 10 == 0:    
            w2sumdistance_dict = distance_experts(wstrid2wnumid, wnumid2wstrid, worker_acc_dict, t2w, t2wl, w2t, path, deepFD)
            
            ##Top1 方法
            # Top1_epoch_list = []
            t2pwld  = Top1_experts(w2sumdistance_dict, wstrid2wnumid, wnumid2wstrid, t2wl)
            acc_Top1, rightlist_Top1, wronglist_Top1  = getaccuracy_Top1(t2truth, t2pwld, t2w)
            
                    
            
            output_list.append(acc_Top1)
            output_list.append(len(rightlist_Top1))
            output_list.append(len(wronglist_Top1))
            
            # Top1_value_list.extend(Top1_epoch_list)
            acc_Top1_all = acc_Top1_all + acc_Top1
            # 20200811 多加的MAX的计算
            if acc_Top1 > max_temp_Top1:
              max_temp_Top1 = acc_Top1

            ## weighted mv 方法
            # WM_epoch_list = []
            mv_t2pwld  = Sort_experts(w2sumdistance_dict, wstrid2wnumid, wnumid2wstrid, t2wl)
            mv_t2lpd = W_MV_AggMethod(mv_t2pwld,label_set)
            acc_mv, rightlist_mv, wronglist_mv  = W_MV_getaccuracy(t2truth, mv_t2lpd, t2w)
            
            output_list.append(acc_mv)
            output_list.append(len(rightlist_mv))
            output_list.append(len(wronglist_mv))
            
            # WM_value_list.extend(WM_epoch_list)
            acc_WM_all = acc_WM_all + acc_mv
            # 20200811 多加的MAX的计算
            if acc_mv > max_temp_MV:
              max_temp_MV = acc_mv
            
            ## 训练完毕,计算最后结果写入文件
            if (epoch+1) % 50 == 0:
              acc_Top1_all = acc_Top1_all / 5
              output_list.append(acc_Top1_all)
              output_list.append(max_temp_Top1)

              acc_WM_all = acc_WM_all / 5
              output_list.append(acc_WM_all)
              output_list.append(max_temp_MV)

              with open(output_file,'a+')as f:
                f_csv = csv.writer(f)
                f_csv.writerow(output_list)
              

流式输出内容被截断，只能显示最后 5000 行内容。
epoch 4 batch 14 loss = 2.4539053
epoch 4 batch 15 loss = 2.3867378
epoch 4 batch 16 loss = 2.4544718
epoch 4 batch 17 loss = 2.487422
epoch 4 batch 18 loss = 2.3173964
epoch 5 batch 0 loss = 2.3350198
epoch 5 batch 1 loss = 2.5833275
epoch 5 batch 2 loss = 2.3196468
epoch 5 batch 3 loss = 2.458021
epoch 5 batch 4 loss = 2.2637415
epoch 5 batch 5 loss = 2.5290709
epoch 5 batch 6 loss = 2.342804
epoch 5 batch 7 loss = 2.4007227
epoch 5 batch 8 loss = 2.1097195
epoch 5 batch 9 loss = 2.524586
epoch 5 batch 10 loss = 2.2316613
epoch 5 batch 11 loss = 2.575252
epoch 5 batch 12 loss = 2.4171696
epoch 5 batch 13 loss = 2.3164313
epoch 5 batch 14 loss = 2.2050302
epoch 5 batch 15 loss = 2.6282492
epoch 6 batch 0 loss = 2.374275
epoch 6 batch 1 loss = 2.4312558
epoch 6 batch 2 loss = 2.5043182
epoch 6 batch 3 loss = 2.6227229
epoch 6 batch 4 loss = 2.3664658
epoch 6 batch 5 loss = 2.4877694
epoch 6 batch 6 loss = 2.371396
epoch 6 batch 7 loss = 2.6546268
epoch 6 batc

In [ ]:
# 20200811 继续测试一下不同embsize 对结果的影响
emb_size_list= [2,3,4,5,6,7,8,9,10]
for emb_size in emb_size_list:
  ## 20200725 弄一下所有的数据集看看结果;
  ## 20200727 看一下mv 失败的task的 结果;
  pathlist = ['/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/data_at_scale/active-crowd-toolkit/CF/',
          '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/data_at_scale/active-crowd-toolkit/CF_amt/',
          '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/data_at_scale/active-crowd-toolkit/MS/',
          '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/data_at_scale/active-crowd-toolkit/SP/',
          '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/data_at_scale/active-crowd-toolkit/SP_amt/',
          '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/data_at_scale/active-crowd-toolkit/ZenCrowd_all/',
          '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/data_at_scale/active-crowd-toolkit/ZenCrowd_in/',
          '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/data_at_scale/active-crowd-toolkit/ZenCrowd_us/',
          '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/data_at_scale/SpectralMethodsMeetEM/bluebird/',
          '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/data_at_scale/SpectralMethodsMeetEM/dog/',
          '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/data_at_scale/SpectralMethodsMeetEM/rte/',
          '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/data_at_scale/SpectralMethodsMeetEM/trec/',
          '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/data_at_scale/SpectralMethodsMeetEM/web/',
          '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/data_at_scale/crowdscale2013/fact_eval/',
          '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/data_at_scale/crowdscale2013/sentiment/',
          '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/data_at_scale/crowd_truth_inference/d_Duck Identification/',
          '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/data_at_scale/crowd_truth_inference/d_jn-product/',
          '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/data_at_scale/crowd_truth_inference/d_sentiment/',
          '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/data_at_scale/crowd_truth_inference/s4_Dog data/',
          '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/data_at_scale/crowd_truth_inference/s4_Face Sentiment Identification/',
          '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/data_at_scale/crowd_truth_inference/s4_Relevance/',
          '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/data_at_scale/crowd_truth_inference/s5_AdultContent/'
            ]
  emb = str(emb_size)
  emb_av_acc = str(emb_size)+'16'+'av_acc'
  max = str(emb_size) +'16'+ 'max_acc'
  headers = ['Method','dataset','epoch_10_acc','Right','Wrong','epoch_20_acc','Right','Wrong','epoch_30_acc','Right','Wrong','epoch_40_acc','Right','Wrong','epoch_50_acc','Right','Wrong',emb_av_acc,max]
  text_top1 = '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/data_at_scale/' + emb +'_16_mv_failed_ISWorker_Top1_acc.csv'
  text_mv = '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/data_at_scale/'+ emb + '_16_mv_failed_ISWorker_MV_acc.csv'


  with open(text_top1,'w')as f:
    f_csv = csv.writer(f)
    f_csv.writerow(headers)
    # f_csv.writerows(rows)

  with open(text_mv,'w')as f:
    f_csv = csv.writer(f)
    f_csv.writerow(headers)

  for path in pathlist:
    seed = 1234
    device = torch.device('cpu')
    # 这几个要调整一下的;
    b_sz = 10
    hidden_size = 128
    emb_size = 4
    epochs = 50
    # Hyper parameters
    alpha = 10
    beta = 20
    gamma = 0.01
    lr = 0.025

    # datafile = path+'answer_3.csv'
    datafile = path + 'answer_mv_failed.csv'
    ########## pb + ab 模式的input
    with open(path+'w2task_input.pickle','rb') as file:
      w2task_input = pickle.load(file)

    # with open(path+'w2w_input.pickle','rb') as file:
    #   w2w_input = pickle.load(file) 
    #######################################################################
    with open(path+'wnumid2wstrid.pickle','rb') as file:
      wnumid2wstrid = pickle.load(file)

    with open(path+'wstrid2wnumid.pickle','rb') as file:
      wstrid2wnumid = pickle.load(file)

    with open(path+'t2w.pickle','rb') as file:
      t2w = pickle.load(file)
    with open(path+'t2wl.pickle','rb') as file:
      t2wl = pickle.load(file)


    with open(path+'t2truth.pickle','rb') as file:
      t2truth = pickle.load(file)


    with open(path+'w2t.pickle','rb') as file:
      w2t = pickle.load(file)

    with open(path+'w2tl.pickle','rb') as file:
      w2tl = pickle.load(file)

    with open(path+'worker_acc_dict.pickle','rb') as file:
      worker_acc_dict = pickle.load(file)



    with open(path+'w2w.pickle','rb') as file:
      w2w = pickle.load(file) 

    with open(path+'training_cps.pickle','rb') as file:
      training_cps = pickle.load(file) 


    with open(path+'label_set.pickle','rb') as file:
      label_set = pickle.load(file)

    # 20200622 worker 之间 的 similarity 和loss 的计算有关系
    # 
    
    dataset = path[78:]
    # rows = []
    csv_data = pd.read_csv(path+'worker_simlarity.csv',low_memory = False,index_col=0)
    worker_similarity_df = pd.DataFrame(csv_data)

    # model 构建和训练
    pb_feat_size = len(t2w)
    w_pb_input = input2numpy(w2task_input,wstrid2wnumid,wnumid2wstrid,pb_feat_size,path,'w_pb_input.npy')
    w_pb_input_tensor = torch.FloatTensor(w_pb_input).to(device)
    
    deepFD = DeepFD(w_pb_input_tensor, w_pb_input_tensor.size(1), emb_size)
    deepFD.to(device)
    model_loss = Loss_DeepFD(w_pb_input_tensor, worker_similarity_df, wnumid2wstrid, training_cps, device, alpha, beta, gamma)

    # 需要写下来的东西, 保存起来的;
    Top1_value_list = []
    acc_Top1_all = 0
    Top1_value_list.append('Top1')
    Top1_value_list.append(dataset)

    WM_value_list = []
    acc_WM_all = 0
    WM_value_list.append('Weight_MV')
    WM_value_list.append(dataset)

    ####20200811多一个max指标
    max_temp_Top1 = 0
    max_temp_MV = 0

    for epoch in range(epochs):
      #logger.info(f'----------------------EPOCH {epoch}-----------------------')
      
      deepFD = train_model(wnumid2wstrid, training_cps, deepFD, model_loss, device, epoch)
      #deepSW = train_model_DeepSW(wnumid2wstrid, deepSW, model_loss, device, epoch)
      
       
      #每训练10次出一个结果;
      if (epoch+1) % 10 == 0:    
        w2sumdistance_dict = distance_experts(wstrid2wnumid, wnumid2wstrid, worker_acc_dict, t2w, t2wl, w2t, path, deepFD)
        
        ##Top1 方法
        Top1_epoch_list = []
        t2pwld  = Top1_experts(w2sumdistance_dict, wstrid2wnumid, wnumid2wstrid, t2wl)
        acc_Top1, rightlist_Top1, wronglist_Top1  = getaccuracy_Top1(t2truth, t2pwld, t2w)
        
                
        
        Top1_epoch_list.append(acc_Top1)
        Top1_epoch_list.append(len(rightlist_Top1))
        Top1_epoch_list.append(len(wronglist_Top1))
        
        Top1_value_list.extend(Top1_epoch_list)
        acc_Top1_all = acc_Top1_all + acc_Top1
        # 20200811 多加的MAX的计算
        if acc_Top1 > max_temp_Top1:
          max_temp_Top1 = acc_Top1

        ## weighted mv 方法
        WM_epoch_list = []
        mv_t2pwld  = Sort_experts(w2sumdistance_dict, wstrid2wnumid, wnumid2wstrid, t2wl)
        mv_t2lpd = W_MV_AggMethod(mv_t2pwld,label_set)
        acc_mv, rightlist_mv, wronglist_mv  = W_MV_getaccuracy(t2truth, mv_t2lpd, t2w)
        
        WM_epoch_list.append(acc_mv)
        WM_epoch_list.append(len(rightlist_mv))
        WM_epoch_list.append(len(wronglist_mv))
        
        WM_value_list.extend(WM_epoch_list)
        acc_WM_all = acc_WM_all + acc_mv
        # 20200811 多加的MAX的计算
        if acc_mv > max_temp_MV:
          max_temp_MV = acc_mv
        
        ## 训练完毕,计算最后结果写入文件
        if (epoch+1) % 50 == 0:
          acc_Top1_all = acc_Top1_all / 5
          Top1_value_list.append(acc_Top1_all)
          Top1_value_list.append(max_temp_Top1)

          acc_WM_all = acc_WM_all / 5
          WM_value_list.append(acc_WM_all)
          WM_value_list.append(max_temp_MV)

          with open(text_top1,'a+')as f:
            f_csv = csv.writer(f)
            f_csv.writerow(Top1_value_list)
            # f_csv.writerow(WM_value_list)
          
          with open(text_mv,'a+')as f:
            f_csv = csv.writer(f)
            # f_csv.writerow(Top1_value_list)
            f_csv.writerow(WM_value_list)


In [ ]:
## 20200725 弄一下所有的数据集看看结果;
## 20200727 看一下mv 失败的task的 结果;
pathlist = ['/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/data_at_scale/active-crowd-toolkit/CF/',
        '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/data_at_scale/active-crowd-toolkit/CF_amt/',
        '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/data_at_scale/active-crowd-toolkit/MS/',
         '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/data_at_scale/active-crowd-toolkit/SP/',
         '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/data_at_scale/active-crowd-toolkit/SP_amt/',
         '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/data_at_scale/active-crowd-toolkit/ZenCrowd_all/',
         '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/data_at_scale/active-crowd-toolkit/ZenCrowd_in/',
         '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/data_at_scale/active-crowd-toolkit/ZenCrowd_us/',
         '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/data_at_scale/SpectralMethodsMeetEM/bluebird/',
         '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/data_at_scale/SpectralMethodsMeetEM/dog/',
         '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/data_at_scale/SpectralMethodsMeetEM/rte/',
         '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/data_at_scale/SpectralMethodsMeetEM/trec/',
         '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/data_at_scale/SpectralMethodsMeetEM/web/',
         '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/data_at_scale/crowdscale2013/fact_eval/',
         '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/data_at_scale/crowdscale2013/sentiment/',
         '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/data_at_scale/crowd_truth_inference/d_Duck Identification/',
         '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/data_at_scale/crowd_truth_inference/d_jn-product/',
         '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/data_at_scale/crowd_truth_inference/d_sentiment/',
         '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/data_at_scale/crowd_truth_inference/s4_Dog data/',
         '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/data_at_scale/crowd_truth_inference/s4_Face Sentiment Identification/',
         '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/data_at_scale/crowd_truth_inference/s4_Relevance/',
         '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/data_at_scale/crowd_truth_inference/s5_AdultContent/'
          ]

headers = ['Method','dataset','epoch_10_acc','Right','Wrong','epoch_20_acc','Right','Wrong','epoch_30_acc','Right','Wrong','epoch_40_acc','Right','Wrong','epoch_50_acc','Right','Wrong','av_acc']
text_top1 = '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/data_at_scale/' + 'mv_failed_ISWorker_Top1_acc.csv'
text_mv = '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/data_at_scale/' + 'mv_failed_ISWorker_MV_acc.csv'


with open(text_top1,'w')as f:
  f_csv = csv.writer(f)
  f_csv.writerow(headers)
  # f_csv.writerows(rows)

with open(text_mv,'w')as f:
  f_csv = csv.writer(f)
  f_csv.writerow(headers)

for path in pathlist:
  seed = 1234
  device = torch.device('cpu')
  # 这几个要调整一下的;
  b_sz = 10
  hidden_size = 128
  emb_size = 4
  epochs = 50
  # Hyper parameters
  alpha = 10
  beta = 20
  gamma = 0.01
  lr = 0.025

  # datafile = path+'answer_3.csv'
  datafile = path + 'answer_mv_failed.csv'
  ########## pb + ab 模式的input
  with open(path+'w2task_input.pickle','rb') as file:
    w2task_input = pickle.load(file)

  # with open(path+'w2w_input.pickle','rb') as file:
  #   w2w_input = pickle.load(file) 
  #######################################################################
  with open(path+'wnumid2wstrid.pickle','rb') as file:
    wnumid2wstrid = pickle.load(file)

  with open(path+'wstrid2wnumid.pickle','rb') as file:
    wstrid2wnumid = pickle.load(file)

  with open(path+'t2w.pickle','rb') as file:
    t2w = pickle.load(file)
  with open(path+'t2wl.pickle','rb') as file:
    t2wl = pickle.load(file)


  with open(path+'t2truth.pickle','rb') as file:
    t2truth = pickle.load(file)


  with open(path+'w2t.pickle','rb') as file:
    w2t = pickle.load(file)

  with open(path+'w2tl.pickle','rb') as file:
    w2tl = pickle.load(file)

  with open(path+'worker_acc_dict.pickle','rb') as file:
    worker_acc_dict = pickle.load(file)



  with open(path+'w2w.pickle','rb') as file:
    w2w = pickle.load(file) 

  with open(path+'training_cps.pickle','rb') as file:
    training_cps = pickle.load(file) 


  with open(path+'label_set.pickle','rb') as file:
    label_set = pickle.load(file)

  # 20200622 worker 之间 的 similarity 和loss 的计算有关系
  # 
  
  dataset = path[78:]
  # rows = []
  csv_data = pd.read_csv(path+'worker_simlarity.csv',low_memory = False,index_col=0)
  worker_similarity_df = pd.DataFrame(csv_data)

  # model 构建和训练
  pb_feat_size = len(t2w)
  w_pb_input = input2numpy(w2task_input,wstrid2wnumid,wnumid2wstrid,pb_feat_size,path,'w_pb_input.npy')
  w_pb_input_tensor = torch.FloatTensor(w_pb_input).to(device)
  
  deepFD = DeepFD(w_pb_input_tensor, w_pb_input_tensor.size(1), emb_size)
  deepFD.to(device)
  model_loss = Loss_DeepFD(w_pb_input_tensor, worker_similarity_df, wnumid2wstrid, training_cps, device, alpha, beta, gamma)

  # 需要写下来的东西, 保存起来的;
  Top1_value_list = []
  acc_Top1_all = 0
  Top1_value_list.append('Top1')
  Top1_value_list.append(dataset)

  WM_value_list = []
  acc_WM_all = 0
  WM_value_list.append('Weight_MV')
  WM_value_list.append(dataset)

  for epoch in range(epochs):
    #logger.info(f'----------------------EPOCH {epoch}-----------------------')
    
    deepFD = train_model(wnumid2wstrid, training_cps, deepFD, model_loss, device, epoch)
    #deepSW = train_model_DeepSW(wnumid2wstrid, deepSW, model_loss, device, epoch)
    
    
    #每训练10次出一个结果;
    if (epoch+1) % 10 == 0:    
      w2sumdistance_dict = distance_experts(wstrid2wnumid, wnumid2wstrid, worker_acc_dict, t2w, t2wl, w2t, path, deepFD)
      
      ##Top1 方法
      Top1_epoch_list = []
      t2pwld  = Top1_experts(w2sumdistance_dict, wstrid2wnumid, wnumid2wstrid, t2wl)
      acc_Top1, rightlist_Top1, wronglist_Top1  = getaccuracy_Top1(t2truth, t2pwld, t2w)
              
      
      Top1_epoch_list.append(acc_Top1)
      Top1_epoch_list.append(len(rightlist_Top1))
      Top1_epoch_list.append(len(wronglist_Top1))
      
      Top1_value_list.extend(Top1_epoch_list)
      acc_Top1_all = acc_Top1_all + acc_Top1

      ## weighted mv 方法
      WM_epoch_list = []
      mv_t2pwld  = Sort_experts(w2sumdistance_dict, wstrid2wnumid, wnumid2wstrid, t2wl)
      mv_t2lpd = W_MV_AggMethod(mv_t2pwld,label_set)
      acc_mv, rightlist_mv, wronglist_mv  = W_MV_getaccuracy(t2truth, mv_t2lpd, t2w)
      
      WM_epoch_list.append(acc_mv)
      WM_epoch_list.append(len(rightlist_mv))
      WM_epoch_list.append(len(wronglist_mv))
      
      WM_value_list.extend(WM_epoch_list)
      acc_WM_all = acc_WM_all + acc_mv
      
      ## 训练完毕,计算最后结果写入文件
      if (epoch+1) % 50 == 0:
        acc_Top1_all = acc_Top1_all / 5
        Top1_value_list.append(acc_Top1_all)
        
        acc_WM_all = acc_WM_all / 5
        WM_value_list.append(acc_WM_all)

        with open(text_top1,'a+')as f:
          f_csv = csv.writer(f)
          f_csv.writerow(Top1_value_list)
          # f_csv.writerow(WM_value_list)
        
        with open(text_mv,'a+')as f:
          f_csv = csv.writer(f)
          # f_csv.writerow(Top1_value_list)
          f_csv.writerow(WM_value_list)



epoch 0 batch 0 loss = 14.415296
epoch 0 batch 1 loss = 13.77039
epoch 0 batch 2 loss = 12.443021
epoch 0 batch 3 loss = 12.132344
epoch 1 batch 0 loss = 11.522124
epoch 1 batch 1 loss = 11.77288
epoch 1 batch 2 loss = 11.527043
epoch 1 batch 3 loss = 10.44588
epoch 2 batch 0 loss = 10.727869
epoch 2 batch 1 loss = 10.318957
epoch 2 batch 2 loss = 10.279736
epoch 3 batch 0 loss = 9.799745
epoch 3 batch 1 loss = 9.88352
epoch 3 batch 2 loss = 10.362183
epoch 3 batch 3 loss = 9.917488
epoch 3 batch 4 loss = 9.771293
epoch 4 batch 0 loss = 10.102808
epoch 4 batch 1 loss = 9.64191
epoch 4 batch 2 loss = 9.833815
epoch 4 batch 3 loss = 9.6812935
epoch 5 batch 0 loss = 9.717601
epoch 5 batch 1 loss = 9.723365
epoch 5 batch 2 loss = 10.370446
epoch 6 batch 0 loss = 9.700329
epoch 6 batch 1 loss = 10.429016
epoch 6 batch 2 loss = 9.742939
epoch 7 batch 0 loss = 9.744229
epoch 7 batch 1 loss = 9.590604
epoch 8 batch 0 loss = 9.948745
epoch 8 batch 1 loss = 9.635075
epoch 8 batch 2 loss = 9.5786

In [ ]:
# #20200702

# w2sumdistance_dict = distance_experts(wstrid2wnumid, wnumid2wstrid, worker_acc_dict, t2w, t2wl, w2t, path, deepFD)
# t2pwld  = Top1_experts(w2sumdistance_dict, wstrid2wnumid, wnumid2wstrid, t2wl)
# # t2lpd = AggMethod(t2pwld,label_set)
# # acc, rightlist, wronglist  = getaccuracy(t2truth, t2lpd, t2w)

# acc, rightlist, wronglist  = getaccuracy_Top1(t2truth, t2pwld, t2w)

# # with open(path+'w2sumdistance_dict.pickle','wb') as file:
# #     pickle.dump(w2sumdistance_dict,file)

# print(acc)
# print(len(wronglist))
# print(len(rightlist))

In [ ]:
# headers = ['Method','dataset','epoch_10_acc','Right','Wrong','epoch_20_acc','Right','Wrong','epoch_30_acc','Right','Wrong','epoch_40_acc','Right','Wrong','epoch_50_acc','Right','Wrong','av_acc']
# text = '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/data_at_scale/' + 'test.csv'
# with open(text,'w')as f:
#   f_csv = csv.writer(f)
#   f_csv.writerow(headers)
#   f_csv.writerow(headers)
#   f_csv.writerow(headers)
#   # f_csv.writerows(rows)

In [ ]:
seed = 1234
# max_vali_f1 = 0 #暂时是没有用到,和分类相关
device = torch.device('cpu')


# 这几个要调整一下的;
b_sz = 10
hidden_size = 128
emb_size = 4
epochs = 10



# Hyper parameters
alpha = 10
beta = 20
gamma = 0.01
lr = 0.025



# path = '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/datasets_fusion_model/s5_AdultContent/'
# path = '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/datasets_fusion_model/s4_Dog data/'
# path = '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/datasets_fusion_model/s4_Relevance/'
# path = '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/datasets_fusion_model/s4_Face Sentiment Identification/'
# path = '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/data_at_scale/active-crowd-toolkit/CF/'
# path = '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/data_at_scale/active-crowd-toolkit/CF_amt/'
path ='/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/data_at_scale/active-crowd-toolkit/MS/'
datafile = path+'answer_3.csv'


#################
# path = '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/truth-inference-at-scale-master/data/SpectralMethodsMeetEM/bluebird/'
# path = '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/truth-inference-at-scale-master/data/SpectralMethodsMeetEM/web/'
# datafile = path+'label.csv'

##### 删掉回答不足5个task的worker;
# path = '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/datasets/s4_Dog data/'
# path = '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/datasets/s4_Relevance/'
# path = '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/datasets/s5_AdultContent/'

# datafile = path+ 'answer_with_condition.csv'

##20200716 hyper question数据集

# path = '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/datasets_fusion_model/hyper-question-datasets/hyper-question-datasets/Science/'
# datafile = path+'new_answer_num.csv'

########## pb + ab 模式的input
with open(path+'w2task_input.pickle','rb') as file:
  w2task_input = pickle.load(file)

# with open(path+'w2w_input.pickle','rb') as file:
#   w2w_input = pickle.load(file) 
#######################################################################

# ############# LLA 模式的input
# with open(path+'w2input_LLA_dict.pickle','rb') as file:
#   w2input_LLA_dict = pickle.load(file)
# with open(path+'w2input_mask_dict.pickle','rb') as file:
#   w2input_mask_dict = pickle.load(file)
#############################################################################


with open(path+'wnumid2wstrid.pickle','rb') as file:
  wnumid2wstrid = pickle.load(file)

with open(path+'wstrid2wnumid.pickle','rb') as file:
  wstrid2wnumid = pickle.load(file)

with open(path+'t2w.pickle','rb') as file:
  t2w = pickle.load(file)
with open(path+'t2wl.pickle','rb') as file:
  t2wl = pickle.load(file)


with open(path+'t2truth.pickle','rb') as file:
  t2truth = pickle.load(file)


with open(path+'w2t.pickle','rb') as file:
  w2t = pickle.load(file)

with open(path+'w2tl.pickle','rb') as file:
  w2tl = pickle.load(file)

with open(path+'worker_acc_dict.pickle','rb') as file:
  worker_acc_dict = pickle.load(file)



with open(path+'w2w.pickle','rb') as file:
  w2w = pickle.load(file) 

with open(path+'training_cps.pickle','rb') as file:
  training_cps = pickle.load(file) 


with open(path+'label_set.pickle','rb') as file:
  label_set = pickle.load(file)

# 20200622 worker 之间 的 similarity 和loss 的计算有关系
# 
csv_data = pd.read_csv(path+'worker_simlarity.csv',low_memory = False,index_col=0)
worker_similarity_df = pd.DataFrame(csv_data)

In [ ]:
# 20200721 wpb input
# 把数据处理成tensor

pb_feat_size = len(t2w)
w_pb_input = input2numpy(w2task_input,wstrid2wnumid,wnumid2wstrid,pb_feat_size,path,'w_pb_input.npy')
w_pb_input_tensor = torch.FloatTensor(w_pb_input).to(device)

In [ ]:
# # lla_input
# # 把数据处理成tensor

# # 20200622 处理成tensor 是要注意维度 LLA 的维度是 task * choice
# pb_feat_size = len(t2w)*len(label_set)
# w_pb_input = input2numpy(w2input_LLA_dict,wstrid2wnumid,wnumid2wstrid,pb_feat_size,path,'w_pb_input.npy')
# w_pb_input_tensor = torch.FloatTensor(w_pb_input).to(device)



# #下面的处理w_ab_input的函数要重新写,把输入换成w2wsim
# ab_feat_size = len(w2w_input)
# w_ab_input = w2wsim2numpy(w2wsim,wstrid2wnumid,wnumid2wstrid,ab_feat_size,path,'w_ab_input.npy')
# #features = torch.FloatTensor(w_pb_input).to(device)
# w_ab_input_tensor = torch.FloatTensor(w_ab_input).to(device)

# w_pb_input = input2numpy(w2task_input,wstrid2wnumid,wnumid2wstrid,pb_feat_size,path,'w_pb_input.npy')
# pb_feat_size = len(t2w)



#### train model


In [ ]:
deepFD = DeepFD(w_pb_input_tensor, w_pb_input_tensor.size(1), emb_size)


deepFD.to(device)
#定义了loss


DeepFD(
  (fc1): Linear(in_features=700, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=4, bias=True)
  (fc3): Linear(in_features=4, out_features=64, bias=True)
  (fc4): Linear(in_features=64, out_features=700, bias=True)
)

In [ ]:
model_loss = Loss_DeepFD(w_pb_input_tensor, worker_similarity_df, wnumid2wstrid, training_cps, device, alpha, beta, gamma)

In [ ]:
#model_loss = Loss_DeepFD(features, getattr(Dl, Dl.ds+'_simi'), args.device, args.alpha, args.beta, args.gamma)



for epoch in range(epochs):
    #logger.info(f'----------------------EPOCH {epoch}-----------------------')
    
    deepFD = train_model(wnumid2wstrid, training_cps, deepFD, model_loss, device, epoch)
    #deepSW = train_model_DeepSW(wnumid2wstrid, deepSW, model_loss, device, epoch)
  

epoch 0 batch 0 loss = 19.600048
epoch 1 batch 0 loss = 18.787388
epoch 2 batch 0 loss = 18.038158
epoch 3 batch 0 loss = 17.381718
epoch 4 batch 0 loss = 16.841234
epoch 5 batch 0 loss = 16.399601
epoch 6 batch 0 loss = 16.011473
epoch 7 batch 0 loss = 15.685134
epoch 8 batch 0 loss = 15.418282
epoch 9 batch 0 loss = 15.346845


### Top1 方法

In [ ]:
#20200702

w2sumdistance_dict = distance_experts(wstrid2wnumid, wnumid2wstrid, worker_acc_dict, t2w, t2wl, w2t, path, deepFD)
t2pwld  = Top1_experts(w2sumdistance_dict, wstrid2wnumid, wnumid2wstrid, t2wl)
# t2lpd = AggMethod(t2pwld,label_set)
# acc, rightlist, wronglist  = getaccuracy(t2truth, t2lpd, t2w)



In [ ]:
t2pwld['109'][0][1]

'7'

In [ ]:
acc, rightlist, wronglist  = getaccuracy_Top1(t2truth, t2pwld, t2w)

# with open(path+'w2sumdistance_dict.pickle','wb') as file:
#     pickle.dump(w2sumdistance_dict,file)

print(acc)
print(len(wronglist))
print(len(rightlist))

0.51
343
357


In [ ]:

# 202007016 距离top3
w2sumdistance_dict = distance_experts(wstrid2wnumid, wnumid2wstrid, worker_acc_dict, t2w, t2wl, w2t, path, deepFD)
t2pwld  = Top1_experts(w2sumdistance_dict, wstrid2wnumid, wnumid2wstrid, t2wl)
t2lpd = AggMethod(t2pwld,label_set)
acc, rightlist, wronglist  = getaccuracy(t2truth, t2lpd, t2w)

# acc, rightlist, wronglist  = getaccuracy_Top1(t2truth, t2pwld, t2w)

# with open(path+'w2sumdistance_dict.pickle','wb') as file:
#     pickle.dump(w2sumdistance_dict,file)

print(acc)
print(len(wronglist))
print(len(rightlist))

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1569: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


0.545067264573991
2029
2431


In [ ]:
# 把wnum_id 转成str_id
def wstr_2_distance(w2sumdistance_dict,wstrid2wnumid):

  # key : wstr_id, value: distance
  wstr2distance = {}
  for wnumid in w2sumdistance_dict:
    wstrid = wnumid2wstrid[wnumid]
    wstr2distance[wstrid] = w2sumdistance_dict[wnumid]

  return wstr2distance

wstr2distance = wstr_2_distance(w2sumdistance_dict,wstrid2wnumid)
with open(path+'wstr2distance.pickle','wb') as file:
    pickle.dump(wstr2distance,file)

### S3 方法
20200709

In [ ]:
#20200702
# w2sumdistance_dict = distance_experts(wstrid2wnumid, wnumid2wstrid, worker_acc_dict, t2w, t2wl, w2t, path, deepFD)
t2s3wl  = S3_experts(wstrid2wnumid, wnumid2wstrid, t2w, t2wl, w2t, path, deepFD)
t2lpd = AggMethod(t2s3wl,label_set)


In [ ]:
acc, rightlist, wronglist  = getaccuracy(t2truth, t2lpd, t2w)

# with open(path+'w2sumdistance_dict.pickle','wb') as file:
#     pickle.dump(w2sumdistance_dict,file)

print(acc)
print(len(wronglist))
print(len(rightlist))

0.3684210526315789
12
7


## weighted_mv
20200720 1_weight = sum(w_ti);

### main 函数

In [ ]:
# 20200720 加入了所有的worker answer ,看一下结果;
w2sumdistance_dict = distance_experts(wstrid2wnumid, wnumid2wstrid, worker_acc_dict, t2w, t2wl, w2t, path, deepFD)
t2pwld  = Sort_experts(w2sumdistance_dict, wstrid2wnumid, wnumid2wstrid, t2wl)
t2lpd = W_MV_AggMethod(t2pwld,label_set)
acc, rightlist, wronglist  = W_MV_getaccuracy(t2truth, t2lpd, t2w)

# acc, rightlist, wronglist  = getaccuracy_Top1(t2truth, t2pwld, t2w)

# with open(path+'w2sumdistance_dict.pickle','wb') as file:
#     pickle.dump(w2sumdistance_dict,file)

print(acc)
print(len(wronglist))
print(len(rightlist))

0.6966666666666667
91
209


## weighted_mv 分两档
20200721 1_weight = sum(w_ti) 分两档或者3挡吧;

In [ ]:
# 20200702 选出top1的worker 作为结果;
def take3rd(elem):
    return elem[2]
# 20200720 对worker 按照distance 排序了 worker 作为结果;
def Sort_experts(w2sumdistance_dict,wstrid2wnumid, wnumid2wstrid, t2wl):

  #t2wl = self.t2wl
  # worker_list = []
  # for i in idnumpy:
  #   strid = wnumid2wstrid[i]
  #   worker_list.append(strid)
  
  # key : task_id, value: wnumid,label,distance
  t2pwld = {}
  for t in t2wl:
    wl_list = t2wl[t]
    w1d_list = []
    predict = 0
    
    for wl_tuple in wl_list:
      worker = wl_tuple[0]
      label = wl_tuple[1]
      numid = wstrid2wnumid[worker]
      w_dis = w2sumdistance_dict[numid]
      w1d_list.append([worker,label,w_dis])
    # 根据 sum_distance  升序排列;
    w1d_list.sort(key=take3rd,reverse=False)
    # 取排在第一位的 wld 信息
    # 20200716 取几个直接进行切片就行了, 不需要进行切片了;
    t2pwld[t] = w1d_list

  return t2pwld

In [ ]:
# def my_sum(i):
#   if i < 0:
#       raise ValueError
#   elif i <= 1:
#       return i
#   else:
#       return i + my_sum(i-1)

# 20200612 为甚要用类写, 不这么写了;

def AggMethod(t2wl,label_set):

  #t2wl = self.t2wl
  t2lpd={}
  for t in t2wl:
    t2lpd[t]={}

    # multi label 初始化;
    for label in label_set:
      t2lpd[t][label] = 0
    # e2lpd[e]['0']=0
    # e2lpd[e]['1']=0
    
    # alls = len(t2wl[t])
    # for item in t2wl[t]:
    #   label=item[1]
    #   t2lpd[t][label]+= 1

    #20200720 加上一个weight;
    wld_list = t2wl[t]
    alls = len(wld_list)
    # weight_sum = my_sum(alls)
    # 分两档的写法;20200721
    weight_sum = alls+ 0.5*alls

    # 分两档的写法;
    for i in range(alls):
      if i < 0.5*alls:
        label = wld_list[i][1]
        weight = 1.0*2
        t2lpd[t][label] += weight
      else:
        label = wld_list[i][1]
        weight = 1.0*1
        t2lpd[t][label] += weight


    # for item in t2wl[t]:
    #   label=item[1]
    #   t2lpd[t][label]+= 1
    
    if alls!=0:
        # e2lpd[e]['0']=1.0*e2lpd[e]['0']/alls
        # e2lpd[e]['1']=1.0*e2lpd[e]['1']/alls
      for label in label_set:
          #20200720 weight_sum
          t2lpd[t][label] = 1.0 * t2lpd[t][label] / weight_sum
    else:
        # e2lpd[e]['0']=0.5
        # e2lpd[e]['1']=0.5
      for label in label_set:
          t2lpd[t][label] = 1.0 / len(label_set)

  # return self.expand(e2lpd)
  return t2lpd

In [ ]:

##20200702

def getaccuracy(e2truth, e2lpd, t2w):
  # 需要确保e2lpd 已经是一个完整结果了; 
  #label 应该是字符串
  count = 0
  right_taskid_list = []
  wrong_taskid_list = []
  # truth = ''
  
  for e in t2w:

    ## 20200627 加上这一句就可以用全部数据集了;
    if e not in e2truth:
      continue
    # print(e)
    temp = 0
    predict_answer = 0
    #找到worker的mv-answer e2lpd 就是各种方法的结果; temp 存了一个值
    for label in e2lpd[e]:
      if temp < e2lpd[e][label]:
        temp = e2lpd[e][label]
        #print(temp)
      
    count += 1 
    for label in e2lpd[e]:
      if temp == e2lpd[e][label]:
        predict_answer = label
      # print('temp',temp)
      # print('predict',predict_answer)
    
    if predict_answer == e2truth[e]:
      
      right_taskid_list.append(e)
    else:
      wrong_taskid_list.append(e)  

    # 准确率, 回答正确的list, 回答错误的list
  return len(right_taskid_list)/count, right_taskid_list, wrong_taskid_list

### main 函数

In [ ]:
# 20200720 加入了所有的worker answer ,看一下结果;
w2sumdistance_dict = distance_experts(wstrid2wnumid, wnumid2wstrid, worker_acc_dict, t2w, t2wl, w2t, path, deepFD)
t2pwld  = Sort_experts(w2sumdistance_dict, wstrid2wnumid, wnumid2wstrid, t2wl)
t2lpd = AggMethod(t2pwld,label_set)
acc, rightlist, wronglist  = getaccuracy(t2truth, t2lpd, t2w)

# acc, rightlist, wronglist  = getaccuracy_Top1(t2truth, t2pwld, t2w)

# with open(path+'w2sumdistance_dict.pickle','wb') as file:
#     pickle.dump(w2sumdistance_dict,file)

print(acc)
print(len(wronglist))
print(len(rightlist))

0.7496902106567535
202
605


## DS_distance
20200721  
Class 原版


In [ ]:
import math
import csv
import random
import sys

class EM:
    def __init__(self,e2wl,w2el,label_set, initquality):
        self.e2wl = e2wl
        self.w2el = w2el
        self.workers = self.w2el.keys()
        self.label_set = label_set
        self.initalquality = initquality
             


# E-step
    # 这个可以直接把distance当做 weight,计算e2lpd
    def Update_e2lpd(self):
        self.e2lpd = {}

        for example, worker_label_set in e2wl.items():
            lpd = {}
            total_weight = 0

            for tlabel, prob in self.l2pd.items():
                weight = prob
                for (w, label) in worker_label_set:
                    weight *= self.w2cm[w][tlabel][label]
                
                lpd[tlabel] = weight
                total_weight += weight
                

            for tlabel in lpd:
                if total_weight == 0:
                    # uniform distribution 
                    lpd[tlabel] = 1.0/len(self.label_set)
                else:
                    lpd[tlabel] = lpd[tlabel]*1.0/total_weight
            
            self.e2lpd[example] = lpd



#M-step

    def Update_l2pd(self):
        # 20200719 等于 0 是为什么?
        for label in self.l2pd:
            self.l2pd[label] = 0
        
        # 20200719 用e2lpd + 0 得到一个新的值
        for _, lpd in self.e2lpd.items():
            for label in lpd:
                self.l2pd[label] += lpd[label]
        # 20200719 这一步在干什么?
        for label in self.l2pd:
            self.l2pd[label] *= 1.0/len(self.e2lpd)


            
    def Update_w2cm(self):

        # 20200719 update之前先定成0;
        for w in self.workers:
            for tlabel in self.label_set:
                for label in self.label_set:
                    self.w2cm[w][tlabel][label] = 0


        w2lweights = {}
        for w in self.w2el:
            w2lweights[w] = {}
            for label in self.label_set:
                w2lweights[w][label] = 0
            for example, _ in self.w2el[w]:
                for label in self.label_set:
                    w2lweights[w][label] += self.e2lpd[example][label]

            
            for tlabel in self.label_set:

                if w2lweights[w][tlabel] == 0:
                    for label in self.label_set:
                        if tlabel == label:
                            self.w2cm[w][tlabel][label] = self.initalquality
                        else:
                            self.w2cm[w][tlabel][label] = (1-self.initalquality)*1.0/(len(self.label_set)-1)

                    continue

                for example, label in self.w2el[w]:
                        #e2lpd[e][t] 就好比是QASCA里面 Qc
                        self.w2cm[w][tlabel][label] += self.e2lpd[example][tlabel]*1.0/w2lweights[w][tlabel]



        return self.w2cm
                    

    
    

                     

#initialization
    def Init_l2pd(self):
        #uniform probability distribution 
        # 最一般的情况, 所有类别初始概率都是一样的;
        l2pd = {}
        for label in self.label_set:
            l2pd[label] = 1.0/len(self.label_set)
        return l2pd

    def Init_w2cm(self):
        w2cm = {}
        for worker in self.workers:
            w2cm[worker] = {}
            for tlabel in self.label_set:
                w2cm[worker][tlabel] = {}
                for label in self.label_set:
                    if tlabel == label:
                        w2cm[worker][tlabel][label] = self.initalquality
                    else:
                        w2cm[worker][tlabel][label] = (1-self.initalquality)/(len(label_set)-1)

        return w2cm

    def Run(self, iterr = 20):
        
        self.l2pd = self.Init_l2pd()
        self.w2cm = self.Init_w2cm()

        while iterr > 0:
            # E-step
            self.Update_e2lpd() 

            # M-step
            self.Update_l2pd()
            self.Update_w2cm()

            # compute the likelihood
            # print self.computelikelihood()

            iterr -= 1
        
        return self.e2lpd, self.w2cm


    def computelikelihood(self):
        
        lh = 0

        for _, worker_label_set in self.e2wl.items():
            temp = 0
            for tlabel, prior in self.l2pd.items():
                inner = prior
                for worker, label in worker_label_set:
                    inner *= self.w2cm[worker][tlabel][label]
                temp += inner
            
            lh += math.log(temp)
        
        return lh


###################################
# The above is the EM method (a class)
# The following are several external functions 
###################################

def getaccuracy(truthfile, e2lpd, label_set):
  e2truth = {}
  f = open(truthfile, 'r')
  reader = csv.reader(f)
  next(reader)

  right_taskid_list = []
  wrong_taskid_list = []

  for line in reader:
      example, truth = line
      e2truth[example] = truth

  tcount = 0
  count = 0

  for e in e2lpd:

    if e not in e2truth:
      # print(e)
      continue

    temp = 0
    for label in e2lpd[e]:
        if temp < e2lpd[e][label]:
            temp = e2lpd[e][label]
    
    candidate = []

    for label in e2lpd[e]:
        if temp == e2lpd[e][label]:
            candidate.append(label)
    
    truth = random.choice(candidate)

    count += 1
      
    ###### 20200721
    if truth == e2truth[e]:
      tcount += 1
      right_taskid_list.append(e)
    else:
      wrong_taskid_list.append(e) 


  return tcount*1.0/count, right_taskid_list, wrong_taskid_list


def gete2wlandw2el(datafile):
    e2wl = {}
    w2el = {}
    label_set=[]
    
    f = open(datafile, 'r')
    reader = csv.reader(f)
    next(reader)

    for line in reader:
        example, worker, label = line
        if example not in e2wl:
            e2wl[example] = []
        e2wl[example].append([worker,label])

        if worker not in w2el:
            w2el[worker] = []
        w2el[worker].append([example,label])

        if label not in label_set:
            label_set.append(label)

    return e2wl,w2el,label_set
    




In [ ]:
import math
import csv
import random
import sys
# 20200721 
class EM:
    def __init__(self,e2wl,w2el,label_set, initquality):
        self.e2wl = e2wl
        self.w2el = w2el
        self.workers = self.w2el.keys()
        self.label_set = label_set
        self.initalquality = initquality
             


# E-step
    # 这个可以直接把distance当做 weight,计算e2lpd
    def Update_e2lpd(self):
        self.e2lpd = {}

        for example, worker_label_set in e2wl.items():
            lpd = {}
            total_weight = 0

            for tlabel, prob in self.l2pd.items():
                weight = prob
                for (w, label) in worker_label_set:
                    weight *= self.w2cm[w][tlabel][label]
                
                lpd[tlabel] = weight
                total_weight += weight
                

            for tlabel in lpd:
                if total_weight == 0:
                    # uniform distribution 
                    lpd[tlabel] = 1.0/len(self.label_set)
                else:
                    lpd[tlabel] = lpd[tlabel]*1.0/total_weight
            
            self.e2lpd[example] = lpd



#M-step

    def Update_l2pd(self):
        # 20200719 等于 0 是为什么?
        for label in self.l2pd:
            self.l2pd[label] = 0
        
        # 20200719 用e2lpd + 0 得到一个新的值
        for _, lpd in self.e2lpd.items():
            for label in lpd:
                self.l2pd[label] += lpd[label]
        # 20200719 这一步在干什么?
        for label in self.l2pd:
            self.l2pd[label] *= 1.0/len(self.e2lpd)


            
    def Update_w2cm(self):

        # 20200719 update之前先定成0;
        for w in self.workers:
            for tlabel in self.label_set:
                for label in self.label_set:
                    self.w2cm[w][tlabel][label] = 0


        w2lweights = {}
        for w in self.w2el:
            w2lweights[w] = {}
            for label in self.label_set:
                w2lweights[w][label] = 0
            for example, _ in self.w2el[w]:
                for label in self.label_set:
                    w2lweights[w][label] += self.e2lpd[example][label]

            
            for tlabel in self.label_set:

                if w2lweights[w][tlabel] == 0:
                    for label in self.label_set:
                        if tlabel == label:
                            self.w2cm[w][tlabel][label] = self.initalquality
                        else:
                            self.w2cm[w][tlabel][label] = (1-self.initalquality)*1.0/(len(self.label_set)-1)

                    continue

                for example, label in self.w2el[w]:
                        #e2lpd[e][t] 就好比是QASCA里面 Qc
                        self.w2cm[w][tlabel][label] += self.e2lpd[example][tlabel]*1.0/w2lweights[w][tlabel]



        return self.w2cm
                    

    
    

                     

#initialization
    def Init_l2pd(self):
        #uniform probability distribution 
        # 最一般的情况, 所有类别初始概率都是一样的;
        l2pd = {}
        for label in self.label_set:
            l2pd[label] = 1.0/len(self.label_set)
        return l2pd

    def Init_w2cm(self):
        w2cm = {}
        for worker in self.workers:
            w2cm[worker] = {}
            for tlabel in self.label_set:
                w2cm[worker][tlabel] = {}
                for label in self.label_set:
                    if tlabel == label:
                        w2cm[worker][tlabel][label] = self.initalquality
                    else:
                        w2cm[worker][tlabel][label] = (1-self.initalquality)/(len(label_set)-1)

        return w2cm

    def Run(self, iterr = 20):
        
        self.l2pd = self.Init_l2pd()
        self.w2cm = self.Init_w2cm()

        while iterr > 0:
            # E-step
            self.Update_e2lpd() 

            # M-step
            self.Update_l2pd()
            self.Update_w2cm()

            # compute the likelihood
            # print self.computelikelihood()

            iterr -= 1
        
        return self.e2lpd, self.w2cm


    def computelikelihood(self):
        
        lh = 0

        for _, worker_label_set in self.e2wl.items():
            temp = 0
            for tlabel, prior in self.l2pd.items():
                inner = prior
                for worker, label in worker_label_set:
                    inner *= self.w2cm[worker][tlabel][label]
                temp += inner
            
            lh += math.log(temp)
        
        return lh


###################################
# The above is the EM method (a class)
# The following are several external functions 
###################################

def getaccuracy(truthfile, e2lpd, label_set):
  e2truth = {}
  f = open(truthfile, 'r')
  reader = csv.reader(f)
  next(reader)

  right_taskid_list = []
  wrong_taskid_list = []

  for line in reader:
      example, truth = line
      e2truth[example] = truth

  tcount = 0
  count = 0

  for e in e2lpd:

    if e not in e2truth:
      # print(e)
      continue

    temp = 0
    for label in e2lpd[e]:
        if temp < e2lpd[e][label]:
            temp = e2lpd[e][label]
    
    candidate = []

    for label in e2lpd[e]:
        if temp == e2lpd[e][label]:
            candidate.append(label)
    
    truth = random.choice(candidate)

    count += 1
      
    ###### 20200721
    if truth == e2truth[e]:
      tcount += 1
      right_taskid_list.append(e)
    else:
      wrong_taskid_list.append(e) 


  return tcount*1.0/count, right_taskid_list, wrong_taskid_list


def gete2wlandw2el(datafile):
    e2wl = {}
    w2el = {}
    label_set=[]
    
    f = open(datafile, 'r')
    reader = csv.reader(f)
    next(reader)

    for line in reader:
        example, worker, label = line
        if example not in e2wl:
            e2wl[example] = []
        e2wl[example].append([worker,label])

        if worker not in w2el:
            w2el[worker] = []
        w2el[worker].append([example,label])

        if label not in label_set:
            label_set.append(label)

    return e2wl,w2el,label_set

### main 函数

In [ ]:
# path = '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/datasets_fusion_model/s5_AdultContent/'
# path = '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/datasets_fusion_model/s4_Dog data/'
path = '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/datasets_fusion_model/s4_Relevance/'
# path = '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/datasets_fusion_model/s4_Face Sentiment Identification/'

datafile = path+'answer.csv'

e2wl,w2el,label_set = gete2wlandw2el(datafile) # generate structures to pass into EM
iterations = 20 # EM iteration number
initquality = 0.7
e2lpd, w2cm = EM(e2wl,w2el,label_set,initquality).Run(iterations)

# datasetname = path[78:]
# print(len(e2lpd))
# print(datasetname)
# print(len(w2cm))
# print(len(e2lpd))

truthfile = path + 'truth.csv'
accuracy, rightlist_ds, wronglist_ds = getaccuracy(truthfile, e2lpd, label_set)
print(accuracy)

0.6130044843049327


In [ ]:
print(accuracy)

0.6130044843049327


In [ ]:
len(rightlist_ds)

2734

In [ ]:
# co_task = set(task_list_w1).union(set(task_list_w2))
#     #回答一致的次数
co_list = set(rightlist_ds).intersection(set(rightlist))

In [ ]:
rightlist_ds

In [ ]:
print(len(rightlist_ds))
print(len(rightlist))
print(len(co_list))

2734
2417
2035


In [ ]:
for w_2 in w2t: 
    task_list_w2 = w2t[w_2]
    task_answer_list_w2 = w2tl[w_2]
    co_task = set(task_list_w1).union(set(task_list_w2))
    #回答一致的次数
    co_answer = set(task_answer_list_w1).intersection(set(task_answer_list_w2))
    
    if len(co_answer) == 0:
      sim = 1/(task_num+len(co_task))
    elif len(co_answer) == len(co_task):
      sim = (len(co_answer)+task_num-1)/(len(co_task)+task_num)
    else:
      sim = (len(co_answer))/(len(co_task))
    wid=wstrid2wnumid[w_2]
    # worker_similarity_df.loc[w1,w2] = sim
    w2wsim_vector[wid] = sim

In [ ]:
# # path = '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/datasets_fusion_model/s4_Dog data/'
# path = '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/datasets_fusion_model/s4_Relevance/'
# # path = '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/datasets_fusion_model/s5_AdultContent/'
# # path = '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/datasets_fusion_model/s4_Face Sentiment Identification/'
# datafile = path+'answer.csv'
###############
#blue birds 数据集
# path = '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/truth-inference-at-scale-master/data/SpectralMethodsMeetEM/bluebird/'
# path = '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/truth-inference-at-scale-master/data/SpectralMethodsMeetEM/web/'
#####

# path = '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/datasets/s4_Dog data/'
# path = '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/datasets/s4_Relevance/'

# pathlist = ['/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/datasets/s4_Relevance/',
#         '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/datasets/s4_Dog data/',
#         '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/datasets/s4_Face Sentiment Identification/',
#         '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/datasets/s5_AdultContent/'] 

# pathlist =['/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/datasets_fusion_model/hyper-question-datasets/hyper-question-datasets/Chinese/',
#         '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/datasets_fusion_model/hyper-question-datasets/hyper-question-datasets/English/',
#          '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/datasets_fusion_model/hyper-question-datasets/hyper-question-datasets/IT/',
#           '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/datasets_fusion_model/hyper-question-datasets/hyper-question-datasets/Medicine/',
#            '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/datasets_fusion_model/hyper-question-datasets/hyper-question-datasets/Pokemon/',
#            '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/datasets_fusion_model/hyper-question-datasets/hyper-question-datasets/Science/'
#  ]
for path in pathlist:
  # datafile = path+'label.csv'
  datafile = path+ 'answer_with_condition.csv'
  # datafile = sys.argv[1]
  e2wl,w2el,label_set = gete2wlandw2el(datafile) # generate structures to pass into EM
  iterations = 20 # EM iteration number
  initquality = 0.7
  e2lpd, w2cm = EM(e2wl,w2el,label_set,initquality).Run(iterations)
  datasetname = path[78:]
  print(len(e2lpd))
  print(datasetname)
  print(len(w2cm))
  print(len(e2lpd))

  truthfile = path + 'truth.csv'
  accuracy = getaccuracy(truthfile, e2lpd, label_set)
  print(accuracy)

IndentationError: ignored

In [ ]:
# #### 对应于hyper 数据集 都很小;
# pathlist =['/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/datasets_fusion_model/hyper-question-datasets/hyper-question-datasets/Chinese/',
#         '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/datasets_fusion_model/hyper-question-datasets/hyper-question-datasets/English/',
#          '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/datasets_fusion_model/hyper-question-datasets/hyper-question-datasets/IT/',
#           '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/datasets_fusion_model/hyper-question-datasets/hyper-question-datasets/Medicine/',
#            '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/datasets_fusion_model/hyper-question-datasets/hyper-question-datasets/Pokemon/',
#            '/content/drive/My Drive/Colab Notebooks/Deep worker cluster/datasets/datasets_fusion_model/hyper-question-datasets/hyper-question-datasets/Science/'
#            ]
# for path in pathlist:
#   # datafile = path+'label.csv'
#   datafile = path+ 'new_answer.csv'
#   # datafile = sys.argv[1]
#   e2wl,w2el,label_set = gete2wlandw2el(datafile) # generate structures to pass into EM
#   e2wl_name ={}
   
#   iterations = 20 # EM iteration number
#   initquality = 0.7
#   e2lpd, w2cm = EM(e2wl,w2el,label_set,initquality).Run(iterations)
#   # print(len(e2lpd))
#   datasetname = path[139:]
#   print(datasetname)
#   print(len(w2cm))
#   print(len(e2lpd))
#   truthfile = path + 'new_truth.csv'
#   accuracy = getaccuracy(truthfile, e2lpd, label_set)
#   print(accuracy)

Chinese/
50
23
0.6086956521739131
English/
63
29
0.4827586206896552
IT/
36
24
0.7083333333333334
Medicine/
45
35
0.6857142857142857
Pokemon/
55
19
0.6842105263157895
Science/
111
19
0.5263157894736842
